# Округление до определённой цифры (усечение до определённой цифры)

TODO: jupyterlab-spellchecker объединение словарей 
[Многоязыковая проверка орфографии для программ, использующих Hunspell](https://habr.com/ru/articles/158441/)
[hunspell-merge](https://github.com/arty-name/hunspell-merge/)

Наверно каждому приходилось когда-то что-то как-то округлять. Обычно задача 
не очень ответственная и решается по месту и на глазок. Хотя и не всегда 
очевидная, по крайней мере, и на русском StackOverflow (ruSO), и на 
английском (SO), уже имеется множество вопросов и множество ответов. Но 
вопросы всё равно продолжают появляться.

Другое дело, зачастую, ответы не слишком уж строгие (по месту и на глазок), 
типа, "Попробуйте так или этак, в вашем случае должно помочь".

А с другой стороны наличие в SQL функций: 
[`TRUNC(n1[, n2])`](https://docs.oracle.com/cd/E11882_01/server.112/e41084/functions221.htm#SQLRF06150) 
[`ROUND(n[, integer])`](https://docs.oracle.com/cd/E11882_01/server.112/e41084/functions155.htm#SQLRF00698), 
как бы, ISO/IEC 9075 серьёзный документ, не филькина грамота, намекает, 
что, во-первых, дело нужное и, во-вторых, может иметь достаточно строгое 
определение. Впрочем и Python 
[`round(number, ndigits=None)`](https://docs.python.org/3/library/functions.html#round)
тоже вполне себе авторитетный документ.

Попробую связанно изложить всё о способах "Округления/усечения до заданной 
цифры" на C/C++/Fortran/Java/Python.

## Об этих заметках (ipynb)

Поскольку на разных платформах поддерживаемые возможности Jupyter Notebook 
различаются:

- На Google Colab нет `math.fma` (Python 3.11);
- У PyCharm, по умолчанию, нет `np.float128` (`np.longdouble == np.double`).

В заметке используются внутренние переменные для возможности тестирования 
этих ограничений.

In [1]:
import decimal
import doctest
import fractions
import IPython.core.magic
import math
import numpy as np
import unittest

# TODO для GitHub тестирования, получить настройки из os.environ
_exr_debug_not_fma = False
_exr_debug_not_float128 = False  # True  # False
_exr_debug_not_fortranmagic = False
_exr_debug_not_Cython = False
_exr_debug_small_dd = False

_exr_float_types = (float, np.half, np.single, np.double)
if _exr_float_types[-1] != np.longdouble and not _exr_debug_not_float128:
    _exr_float_types += (np.longdouble, )
_exr_roundings = [
    decimal.ROUND_CEILING,
    decimal.ROUND_DOWN,
    decimal.ROUND_FLOOR,
    decimal.ROUND_HALF_EVEN,
    decimal.ROUND_HALF_UP,
    decimal.ROUND_HALF_DOWN,
    decimal.ROUND_UP,
    decimal.ROUND_05UP,
]

if not _exr_debug_not_float128 and 'float128' in np.__dict__:
    if np.longdouble != np.float128:
        print("Восстанавливаем np.longdouble после предыдущего")
        np.longdouble = np.float128
else:
    if np.longdouble != np.double:
        print("Устанавливаем np.longdouble = np.double, для полной имитаций")
        np.longdouble = np.double

### Примеры Фортран

Используются ячейки `%fortran` ([Fortran magic](https://github.com/mgaitan/fortran_magic)), 
так же, ячейки зависящие от `%fortran` ячеек отмечаются как `%fortran_depended`.

In [2]:
if _exr_debug_not_fortranmagic:
    _extr_fortranmagic = False
else:
    try:
        %load_ext fortranmagic
        _extr_fortranmagic = True
    except ModuleNotFoundError:
        if 'google.colab' not in str(get_ipython()):
            _extr_fortranmagic = False
            print("fortran-magic не установлен, %fortran ячейки пропускаются")
            @IPython.core.magic.register_cell_magic
            def fortran(line, cell):
                print("Ячейка %fortran пропущена")
        else:
            # Install required packages
            !apt-get update -qq
            !apt-get install -qq gfortran
            !pip install -qq fortran-magic
            # Обход проблемы  установки Numpy в Google Colab
            # https://github.com/mgaitan/fortran_magic/issues/39
            !pip install -qq meson charset-normalizer ninja cmake pkgconfig
            %load_ext fortranmagic
            %fortran_config --backend meson -v

@IPython.core.magic.register_cell_magic
def fortran_depended(line, cell):
    global _extr_fortranmagic
    if _extr_fortranmagic:
        get_ipython().run_cell(cell)
    else:
        print("Ячейка %fortran_depended пропущена")

In [6]:
%%fortran

subroutine f1(x, y, z)
    real, intent(in) :: x,y
    real, intent(out) :: z

    z = sin(x+y)

end subroutine f1


Ok. The following fortran objects are ready to use: f1


In [7]:
%%fortran_depended
%precision %g
f1(1.0, 2.1415)

9.26574e-05

### Примеры С++

TODO: использовать pybind11/cppimport или Cython?

В colab Cython есть из коробки

[`%pybind11`](https://github.com/aldanor/ipybind) - 8 years ago

Вариант,  [bp_magic](https://github.com/abingham/boost_python_tutorial?tab=readme-ov-file#the-bp_magicpy-extension)



In [3]:
if _exr_debug_not_Cython:
    _extr_cythonmagic = False
else:
    try:
        %load_ext Cython
        _extr_cythonmagic = True
    except ModuleNotFoundError:
        _extr_cythonmagic = False
        print("cython не установлен, %cython ячейки пропускаются")
        @IPython.core.magic.register_cell_magic
        def cython(line, cell):
            print("Ячейка %cython пропущена")

@IPython.core.magic.register_cell_magic
def cython_depended(line, cell):
    global _extr_cythonmagic
    if _extr_cythonmagic:
        get_ipython().run_cell(cell)
    else:
        print("Ячейка %cython_depended пропущена")

In [4]:
%%cython
def f(x):
    return 3.0*x

In [5]:
%%cython_depended
f(3.34)

10.02

## Округление Python

Начнём с Python, он, конечно, крайний по алфавиту, зато в деле 
округления/усечения у него, прямо в стандартной библиотеке, неплохой выбор.

### Функция `round()` и округление произвольных типов


Стандартная функция [`round(number, ndigits=None)`](https://docs.python.org/3/library/functions.html#round) 
вызывает магический метод [`object.__round__(self[, ndigits])`](https://docs.python.org/3/reference/datamodel.html#object.__round__), 
который обеспечивает округление до ближайшего чётного по заданной цифре 
в машинном представлении. Однако, алгоритм и точность выполнения этой 
функции зависит от конкретного типа, к примеру, хотя двоичное представление 
типов `float` и `np.double` совпадают, `round(np.double(0.0115), 3)` 
вычислит не очень точное значение, примерно идентичное 
`round(0.0115*1000)/1000`, которое отличается от точного результата 
`round(0.0115, 3)`.

Округление в машинном представлении это общепринятая практика, форматные 
преобразования работают точно также: (`round(x, n) == float(f"{x:.{n}f}")`), 
но в документации этот небольшой сюрприз отдельно разобран: 
`round(2.675, 2) == 2.67`, поскольку `float` имеет двоичное представление, но 
`float(round(decimal.Decimal('2.675'), 2)) == 2.68` или 
`float(round(fractions.Fraction('2.675'), 2)) == 2.68`.

Но настоящий сюрпризище, я бы сказал, даже грабли, в том, что в 
C/C++/SQL/... **одноимённая(!)** функция 
[`ROUND(n[, integer])`](https://docs.oracle.com/cd/E11882_01/server.112/e41084/functions155.htm#SQLRF00698) 
округляет иначе, до ближайшего большего по модулю.

### Швейцарский нож для огругления/усечения

Методы `decimal.Decimal(x).to_integral_value()` или 
`decimal.Decimal(x).quantize()` позволяют выполнять практически все 
мыслимые методы округления/усечения для основных типов данных. 

В отличие от функции `round()` реализация алгоритмов округления собственная,
в общем, не уступающая по занудности и подробности реализации 
`object.__round__()`.

In [6]:
def exr_easy_round(x: 'Union[float, decimal.Decimal, str, np.double]', 
                   ndigits=0, rounding=decimal.ROUND_HALF_UP, fmt_out=None
                  ) -> 'type(x)':
    """Краткий макет "улучшения" стандартного `round(x, ndigits=None)`.
    """ 
    dx = decimal.Decimal(x if fmt_out else str(x))
    return type(x)(dx.scaleb(ndigits).to_integral_value(rounding=rounding).
                      scaleb(-ndigits))
    
    # Для ndigits + log10(abs(x)) < getcontext().prec можно немного короче и
    # слегка шустрее:
    # return type(x)(dx.quantize(decimal.Decimal(f"1e{-ndigits}"),
    #                            rounding=rounding))
    # Но тогда больше ограничений и слов в описании

#### Описание простого интерфейса `exr_easy_round()`

In [7]:
_exr_common_round_docstring = """
    Без опциональных аргументов соответствует функции C/C++/SQL/... `round()`.
    При задании аргумента `rounding=decimal.ROUND_DOWN` соответствует функции 
    SQL `TRUNC(n1[, n2])`. При задании аргументов 
    `rounding=decimal.ROUND_HALF_EVEN, fmt_out=True` соответствует стандартной 
    функции Python `round()`.

    Параметры
    ---------
    x:
        Входные данные
    ndigits:
        Требуемое число знаков после запятой (или до запятой, если <0). Для 
        стандартных двоичных типов `x` с плавающей запятой, обычно, не очень  
        осмысленно превышать по модулю: `sys.float_info.dig` (или 
        `np.finfo(x).precision`)
    rounding:
        ROUND_CEILING   - округление к большему, аналогично `math.ceil()`, 
                          IEEE 754: roundTowardNegative
        ROUND_DOWN      - округление к меньшему по модулю, аналогичен 
                          `math.trunc()` и `int()`, IEEE 754: 
                          roundTowardZero (усечение к 0)
        ROUND_FLOOR     - округление к меньшему, аналогично `math.floor()`, 
                          IEEE 754: roundTowardNegative
        ROUND_HALF_EVEN - округление до ближайшего чётного, аналогично 
                          стандартной функции Python `round()`, 
                          IEEE 754: roundTiesToEven (округление банкира)
        ROUND_HALF_UP   - округление до ближашего большего по модулю, 
                          аналогично функции C/C++/SQL/... `round()`, 
                          IEEE 754: roundTiesToAway (математическое 
                          округление)
        ROUND_HALF_DOWN - округление до ближайшего меньшего по модулю, 
                          широкораспространённых аналогов не имеет, 
                          в IEEE 754 не входит
        ROUND_UP        - округление к большему по модулю, 
                          широкораспространённых аналогов не имеет, 
                          в IEEE 754 не входит
        ROUND_05UP      - округление к меньшему по модулю, если последняя 
                          цифра результата не получается 0 или 5, в этом 
                          случае округление к большему по модулю,
                          широкораспространённых аналогов не имеет, 
                          в IEEE 754 не входит
    fmt_out:
        `True` - точно округляется/усекается машинное представление, см. 
        ниже "Возвращаемое значение";
        
        `False` - требуется точное округление человекочитаемого 
        представления;
        
        Если не задан, используется наиболее быстрый метод, возможно, 
        неразличающий эти случаи, например, ввиду ошибок округления или 
        иных причин. Примерно аналогично функции `numpy.round(a, 
        decimals=0, out=None)`

    Возвращаемое значение
    ---------------------
        Если аргумент `ndigits` не задан (`None`), то выполняется 
        округление к целому (`fmt_out` игнорируется, ввиду совпадения 
        представлений), возвращается результат типа `int`

        Если `fmt_out == True`, то `exr_round(...)` является машинным 
        представлением округлённого машинного представления `x`, что 
        аналогично форматному выводу большинства языков программирования, 
        а так же стандартной функции Python `round()` (машинное 
        представление зависит от типа аргумента, у `float` и `numpy.double` 
        - двоичное, а у `decimal.Decimal`, `str` - десятичное).

        Если `fmt_out == False`, то `str(exr_round(...))` является 
        округлённым значением `str(x)`, т.е. округляем человекочитаемое 
        представление `x`. Может применяться, например, при округлении 
        входных данных и т.п.

        Для методов ROUND_FLOOR и ROUND_DOWN гарантируется, что результат не 
        больше (абсолютно или, соответственно, по модулю) входного значения. 
        Аналогично, для методов ROUND_CEILING и ROUND_UP, не меньше.
"""

exr_easy_round.__doc__ += _exr_common_round_docstring + """
        В данном макете, по умолчанию, округляется человекочитаемое 
        представление, поскольку `Decimal(float(x))` почти наполовину 
        забит "ошибками" округления двоичного значения `float` (само 
        преобразование, конечно, точное, но 2**-52 имеет 53 знака после 
        запятой), `Decimal(str(x))` заметно эффективнее.

        Возврат значения типа аргумента или типа `int` в зависимости от 
        `ndigits != None`, аналогичного стандартному `round()`, в макете 
        не реализован.

    >>> import decimal
    >>> import numpy as np
    >>>
    >>> exr_easy_round(2.0115, 3)
    2.012
    >>> exr_easy_round(2.0115, 3, fmt_out=True)
    2.011
    >>> exr_easy_round(2.0115, 3, fmt_out=False)
    2.012
    >>> exr_easy_round(20115, -1)
    20120
    >>> exr_easy_round(decimal.Decimal("2.0115"), 3, fmt_out=True)
    Decimal('2.012')
    >>> exr_easy_round(decimal.Decimal("2.0115"), 3, fmt_out=False)
    Decimal('2.012')
    >>> exr_easy_round(np.double(4.00001), 5, rounding=decimal.ROUND_DOWN, 
    ...                fmt_out=True)
    np.float64(4.0)
    >>> exr_easy_round(np.double(4.00001), 5, rounding=decimal.ROUND_DOWN, 
    ...                fmt_out=False)
    np.float64(4.00001)
    >>> exr_easy_round("4.00001", 5, rounding=decimal.ROUND_DOWN, 
    ...                fmt_out=True)
    '4.00001'
    >>> exr_easy_round("4.00001", 5, rounding=decimal.ROUND_DOWN, 
    ...                fmt_out=False)
    '4.00001'
    >>> with decimal.localcontext(prec=29):
    ...     print(exr_easy_round("2.71828182845904523536028747135266e25", 3))
    27182818284590452353602874.714
    """

doctest.testmod()

**********************************************************************
File "__main__", line 104, in __main__.exr_easy_round
Failed example:
    exr_easy_round(np.double(4.00001), 5, rounding=decimal.ROUND_DOWN, 
                   fmt_out=True)
Expected:
    np.float64(4.0)
Got:
    4.0
**********************************************************************
File "__main__", line 107, in __main__.exr_easy_round
Failed example:
    exr_easy_round(np.double(4.00001), 5, rounding=decimal.ROUND_DOWN, 
                   fmt_out=False)
Expected:
    np.float64(4.00001)
Got:
    4.00001
**********************************************************************
1 items had failures:
   2 of  13 in __main__.exr_easy_round
***Test Failed*** 2 failures.


TestResults(failed=2, attempted=13)

#### Более сложный интерфейс `exr_decimal_round()` (эталон для тестов)

In [8]:
def exr_decimal_round(x: 'Union[float, decimal.Decimal, str, ' 
                               'npt.NDArray[tuple[]], np.half, ' 
                               'np.single, np.double, np.longdouble]', 
                      ndigits=None, rounding=decimal.ROUND_HALF_UP, 
                      fmt_out=None
                     ) -> 'Union[int, type(x)]':
    """`Decimal` макет "улучшения" стандартного `round(x, ndigits=None)`.
    """
    x_to = type(x)
    if x_to == np.ndarray:
        x_to = x.dtype.type
        x = x_to(x)
    if fmt_out != True:
        pre_cvt_x = str(x)
        if x_to == _exr_dr_float128_t:
            x_to = exr_decimal_to_longdouble
    elif x_to in _exr_dr_decimal_accepted_types:
        pre_cvt_x = x
    elif x_to == _exr_dr_float128_t:
        pre_cvt_x = exr_decimal_from_longdouble(x)
        x_to = exr_decimal_to_longdouble
    else:
        pre_cvt_x = float(x)
    dx = decimal.Decimal(pre_cvt_x)
    if ndigits is None:
        return int(dx.to_integral_value(rounding=rounding))
    return x_to(dx.scaleb(ndigits).to_integral_value(rounding=rounding).
                   scaleb(-ndigits))


if "float128" not in np.__dict__  or   _exr_debug_not_float128:
    # Случай, когда np.longdouble == np.double 
    # (ARM, PyCharm на x86, по умолчанию, ...)
    _exr_dr_use_float128 = False
    _exr_doctest_longdouble = "doctest: +SKIP"
else:
    _exr_dr_use_float128 = True
    _exr_doctest_longdouble = ""

exr_decimal_round.__doc__ += _exr_common_round_docstring + f"""
        В данном макете, по умолчанию, округляется человекочитаемое 
        представление, поскольку `Decimal(float(x))` почти наполовину 
        забит "ошибками" округления двоичного значения `float` (само 
        преобразование, конечно, точное, но 2**-52 имеет 53 знака после 
        запятой), `Decimal(str(x))` заметно эффективнее.

        Для `np.array` поддерживаются только 0-мерные массивы.

    >>> import decimal
    >>> import numpy as np
    >>>
    >>> exr_decimal_round(2.0115, 3, fmt_out=True)
    2.011
    >>> exr_decimal_round(2.0115, 3, fmt_out=False)
    2.012
    >>> exr_decimal_round(decimal.Decimal("2.0115"), 3, fmt_out=True)
    Decimal('2.012')
    >>> exr_decimal_round(decimal.Decimal("2.0115"), 3, fmt_out=False)
    Decimal('2.012')
    >>> exr_decimal_round(np.double(4.00001), 5, rounding=decimal.ROUND_DOWN, 
    ...                   fmt_out=True)
    np.float64(4.0)
    >>> exr_decimal_round(np.double(4.00001), 5, rounding=decimal.ROUND_DOWN, 
    ...                   fmt_out=False)
    np.float64(4.00001)
    >>> exr_decimal_round("4.00001", 5, rounding=decimal.ROUND_DOWN, 
    ...                   fmt_out=True)
    '4.00001'
    >>> exr_decimal_round("4.00001", 5, rounding=decimal.ROUND_DOWN, 
    ...                   fmt_out=False)
    '4.00001'
    >>> exr_decimal_round("inf", 0)
    'Infinity'
    >>> exr_decimal_round("nan", 0)
    'NaN'
    >>> exr_decimal_round(2012.5, 0)
    2013.0
    >>> exr_decimal_round(2012.5)
    2013
    >>> exr_decimal_round(np.longdouble("0.222222222222222225"), 17,  # {_exr_doctest_longdouble}
    ...                   fmt_out=True)
    np.longdouble('0.22222222222222222')
    >>> exr_decimal_round(np.longdouble("0.222222222222222225"), 17,  # {_exr_doctest_longdouble} 
    ...                   fmt_out=False)
    np.longdouble('0.22222222222222223')
    >>> a0d = np.array(np.longdouble('3156294995342131.5065'))
    >>> exr_decimal_round(a0d, 3)                               # {_exr_doctest_longdouble}
    np.longdouble('3156294995342131.507')
    """

_exr_dr_1 = decimal.Decimal(1)
_exr_dr_decimal_accepted_types = (float, np.double, decimal.Decimal, str)
if not _exr_dr_use_float128:
    _exr_dr_float128_t = type(None)
    _exr_dr_float128_types = tuple()
    exr_decimal_from_longdouble = decimal.Decimal.from_float
else:
    _exr_dr_float128_t = np.float128
    _exr_dr_float128_types = (_exr_dr_float128_t, np.ndarray)
    _exr_dr_max_digits10 = np.finfo(_exr_dr_float128_t).precision + 2
    
    def exr_decimal_from_longdouble(ld: np.longdouble, 
                                    prec=None) -> decimal.Decimal:
        """
        Converts a `np.longdouble` to a decimal number, exactly.
        Since 0.1 is not exactly representable in binary floating point,
        `exr_decimal_from_longdouble(np.longdouble('0.1'))` is not the same 
        as `Decimal('0.1')`.

        Parameters
        ----------
        ld
            Input `np.longdouble`
        prec
            If exist, reqired binary to decimal precision. By default, 
            an accurate decimal representation is provided, which may 
            require precision up to 11514 digits

        Returns
        -------
        decimal.Decimal
             Decimal representation of `ld`
             
        >>> import numpy as np
        >>>
        >>> exr_decimal_from_longdouble(
        ...     np.longdouble('0.1'))  # doctest: +ELLIPSIS
        Decimal('0.1000000000000000000013552527156...5625')
        >>> exr_decimal_from_longdouble(
        ...     np.exp(np.longdouble(1000)))  # doctest: +ELLIPSIS 
        Decimal('19700711140...3200')
        >>> exr_decimal_from_longdouble(0.0)
        Decimal('0')
        >>> exr_decimal_from_longdouble(-0.0)
        Decimal('-0')
        >>> exr_decimal_from_longdouble(np.nan)
        Decimal('NaN')
        >>> exr_decimal_from_longdouble(np.longdouble('Infinity'))
        Decimal('Infinity')
        >>> exr_decimal_from_longdouble(-np.inf)
        Decimal('-Infinity')

        See also: `decimal.Decimal.from_float`
        """
        if not (np.isfinite(ld) and ld):
            return decimal.Decimal(float(ld))
        if prec is None:
            # abs(np.spacing(ld))
            aulp = abs(ld - np.nextafter(ld, 0.))
            lfp = math.ceil(max(0, np.log10(abs(ld)) - 
                                np.log2(aulp) + 1))
            lip = math.ceil(max(0, np.log10(abs(ld))))
            prec = lfp + lip + 4
        with decimal.localcontext(prec=prec):
            ild = ld.as_integer_ratio()
            return decimal.Decimal(ild[0])/decimal.Decimal(ild[1])
        
def exr_decimal_to_longdouble(d) -> np.longdouble:
    """
    Преобразует `Decimal` в `np.longdouble`. 
    
    Примечание: в зависимости от системы и/или от параметров сборки Numpy 
    тип `np.longdouble`, как может иметь точность 64-бит (x86/amd64) или 
    112-бит (IBM Power9, Sparc), так и может и совпадать с типом 
    `np.double` (ARM или сборки Numpy по умолчанию входящие в PyCharm на 
    всех платформах).

    >>> import decimal
    >>>
    >>> exr_decimal_to_longdouble(decimal.Decimal(2).sqrt())     # doctest: +ELLIPSIS
    np...(...1.414213562373095...)
    >>> exr_decimal_to_longdouble(decimal.Decimal('NaN'))        # doctest: +ELLIPSIS
    np...(...nan...)
    >>> exr_decimal_to_longdouble(decimal.Decimal('Infinity'))   # doctest: +ELLIPSIS
    np...(...inf...)
    >>> exr_decimal_to_longdouble(decimal.Decimal('-Infinity'))  # doctest: +ELLIPSIS
    np...(...-inf...)
    
    """
    return np.longdouble(str(d) if d.is_finite() else 
                         float(d))

def exr_approximate_decimal_from_integer_ratio(
                x: 'Union[tuple[int, int], Rational]', 
                prec=None) -> decimal.Decimal:
    n, d = x if isinstance(x, tuple) else x.as_integer_ratio()
    with decimal.localcontext(prec=(prec if prec else
                                    decimal.getcontext().prec + 4)):
        return decimal.Decimal(n)/decimal.Decimal(d)

def exr_decimal_to_fraction(d):
    return fractions.Fraction(*d.as_integer_ratio())

doctest.testmod()

**********************************************************************
File "__main__", line 98, in __main__.exr_decimal_round
Failed example:
    exr_decimal_round(np.double(4.00001), 5, rounding=decimal.ROUND_DOWN, 
                      fmt_out=True)
Expected:
    np.float64(4.0)
Got:
    4.0
**********************************************************************
File "__main__", line 101, in __main__.exr_decimal_round
Failed example:
    exr_decimal_round(np.double(4.00001), 5, rounding=decimal.ROUND_DOWN, 
                      fmt_out=False)
Expected:
    np.float64(4.00001)
Got:
    4.00001
**********************************************************************
File "__main__", line 118, in __main__.exr_decimal_round
Failed example:
    exr_decimal_round(np.longdouble("0.222222222222222225"), 17,  # 
                      fmt_out=True)
Expected:
    np.longdouble('0.22222222222222222')
Got:
    0.22222222222222222
*******************************************************************

TestResults(failed=11, attempted=44)

In [9]:
exr_decimal_round?

Signature:
exr_decimal_round(
    x: 'Union[float, decimal.Decimal, str, npt.NDArray[tuple[]], np.half, np.single, np.double, np.longdouble]',
    ndigits=None,
    rounding='ROUND_HALF_UP',
    fmt_out=None,
) -> 'Union[int, type(x)]'
Docstring:
`Decimal` макет "улучшения" стандартного `round(x, ndigits=None)`.

Без опциональных аргументов соответствует функции C/C++/SQL/... `round()`.
При задании аргумента `rounding=decimal.ROUND_DOWN` соответствует функции 
SQL `TRUNC(n1[, n2])`. При задании аргументов 
`rounding=decimal.ROUND_HALF_EVEN, fmt_out=True` соответствует стандартной 
функции Python `round()`.

Параметры
---------
x:
    Входные данные
ndigits:
    Требуемое число знаков после запятой (или до запятой, если <0). Для 
    стандартных двоичных типов `x` с плавающей запятой, обычно, не очень  
    осмысленно превышать по модулю: `sys.float_info.dig` (или 
    `np.finfo(x).precision`)
rounding:
    ROUND_CEILING   - округление к большему, аналогично `math.ceil()`, 
            

##### Спойлер тестов

In [10]:
def _exr_nextafters(x, steps):
    if np.isnan(x):
        return [x]
    t = type(x)
    r = [ x ]
    for _ in range(steps):
        n = t(np.nextafter(r[0], -np.inf))
        if n != r[0]:
            r.insert(0, n)
        n = t(np.nextafter(r[-1], np.inf))
        if n != r[-1]:
            r.append(n)
    if x:
        r += [-y for y in r]
    else:
        r += [-x]
    return r

def _exr_defaults(a, dflt):
    if a is None:
        return dflt  #if hasattr(a, '__iter__') else [dflt]
    elif hasattr(a, '__iter__') and not isinstance(a, str):  # TODO
        return a
    return [a]

def _exr_call(f, *args, **kwargs):
    return f(*args, **{arg: val for arg, val in kwargs.items() if val is not None})

class _exr_diff_collect:
    def __init__(self):
        self.max_diff = 0
        self.max_case = None
    def add(self, q, std, chk, x, args):
        if not np.isfinite(q):
            print(f"exr_diff_collect.add({q=}, {std=}, {chk=}, {x=}, {args=})")
        diff = q*abs(std - chk)
        if diff > self.max_diff:
            self.max_diff = diff
            self.max_case = dict(std=std, chk=chk, x=x, args=args)
    def __repr__(self):
        return f"exr_diff_collect(max_diff={self.max_diff}, max_case={self.max_case})"

_exr_numbers_from_text = [
    '2.675', '2.0115', '20115', '4.00001', '2013.0', "0.222222222222222225",
    "2.71828182845904523536028747135266e25", '56294995342131.5', 
    '16.055', '16.085', 
    # Умножение?
    '8589934591.059999', '2147483647.8799999', '536870911.96999997',
    '268435455.76999998', '67108863.949999996',
    # Деление
    '524287.9999291525', '524287.9999291524', 
    '262143.99998263217', '262143.9999826321',
    '2.654', '1.9', '0.21', '2.01', '201.0', '0.55555555555555555555555', 
    "0.15", "0.145", "0.0000005", 
    '249.99999999999997', '5.465', '0.00261', '0.00161', str(0.15+0.005*62),
    '2.0125', '0.05', '0.25', '0.005', '0.015', '0.025', '0.0115'
]
_exr_numbers_from_text.sort()
for x, y in zip(_exr_numbers_from_text[:-1], _exr_numbers_from_text[1:]):
    assert x < y, f"Duplicate {x, y=}"
_exr_numbers_from_text += [float(x) for x in _exr_numbers_from_text]

def exr_check_floats(f, types_x=None, test_xs=None, test_ndigits=None, 
                     test_roundings=None, test_fmt_outs=None, 
                     nsteps=16, zero_diff=True, inf_nan=True, 
                     verbose=0,
                     standard=exr_decimal_round):
    ndigits_none_exceptions = (OverflowError, ValueError)
    idn = True
    cnt = {'total': 0, 'except': 0, 'diff': 0, 'type': 0, 'zdiff': 0}
    abs_diff = _exr_diff_collect()
    ndigit_diff = _exr_diff_collect()
    ulp_diff = _exr_diff_collect()

    def fail_verbose(label, diff):
        nonlocal idn, verbose, chk, std, x, chk_args
        if diff:
            idn = False
            cnt[label] += 1
            if verbose:
                print(f"{label}: {chk, std, x=} {chk_args=}")
                verbose -= 1
            
    for t in _exr_defaults(types_x, _exr_float_types):
        max_spacing = np.finfo(t).max
        min_inversible = 1/np.finfo(t).max
        for x0 in _exr_defaults(test_xs, 
                                (list(np.arange(0., 4.01, 1/16))
                                 + [str(a) + str(b) + k*'0' + '.' 
                                    + k*'0' + str(a) + str(b)
                                    for a in range(10)
                                    for b in range(0, 10, 5)
                                    for k in range(3)]
                                 + ([np.inf, np.nan] if inf_nan else [])
                                 #+ _exr_numbers_from_text
                                )):
            for x in _exr_nextafters(t(x0), nsteps):
                for ndigits in _exr_defaults(test_ndigits, 
                                             [None] + list(range(-3, 4, 1))):
                    for rounding in _exr_defaults(test_roundings, 
                                                  _exr_roundings):
                        for fmt_out in _exr_defaults(test_fmt_outs,
                                                     [None, True, False]):
                            try:
                                std = standard(x, ndigits=ndigits, 
                                               rounding=rounding, 
                                               fmt_out=fmt_out)
                            except ndigits_none_exceptions as e:
                                std = e
                                if ndigits is not None  or  np.isfinite(x):
                                    assert False, (
                                        f"{std, x=} {ndigits, rounding, fmt_out, standard=}"
                                    )
                            chk_args = {
                                'ndigits': ndigits
                            }
                            if not isinstance(test_roundings, str):
                                chk_args['rounding'] = rounding
                            if (test_fmt_outs is None or 
                                hasattr(test_fmt_outs, '__iter__')):
                                chk_args['fmt_out'] = fmt_out
                            try:
                                chk = _exr_call(f, x, **chk_args)
                            except ndigits_none_exceptions as e:
                                chk = e
                                if ndigits is not None  or  np.isfinite(x):
                                    fail_verbose("except", True)
                                continue
                            cnt["total"] += 1
                            if type(chk) != type(std):
                                fail_verbose("type", True)
                            if (chk or std):
                                diff = (not (chk == std) and
                                        not (isinstance(chk, _exr_float_types) and
                                             np.isnan(chk) and np.isnan(std)))
                                fail_verbose("diff", diff)
                            else:
                                diff = not ((chk == std) and 
                                            (str(chk) == str(std)))
                                fail_verbose("zdiff", diff)
                            if diff:
                                idn = False
                                abs_diff.add(1, std, chk, x, chk_args)
                                ndigit_diff.add(10**(ndigits if ndigits else 0), 
                                                std, chk, x, chk_args)
                                if (abs(x) < max_spacing and
                                    abs(np.spacing(x)) > min_inversible):
                                    ulp_diff.add(t(1.)/np.spacing(x), 
                                                 std, chk, x, chk_args)
    return dict(idn=idn, cnt=cnt, 
                abs_diff=abs_diff, ndigit_diff=ndigit_diff, ulp_diff=ulp_diff)

exr_check_floats(round, test_roundings=decimal.ROUND_HALF_EVEN, 
                 test_fmt_outs=True, inf_nan=False,
                 types_x=float,
                 verbose=10)
# exr_check_floats(round, test_roundings=decimal.ROUND_HALF_EVEN, test_fmt_outs=True)

{'idn': True,
 'cnt': {'total': 64976, 'except': 0, 'diff': 0, 'type': 0, 'zdiff': 0},
 'abs_diff': exr_diff_collect(max_diff=0, max_case=None),
 'ndigit_diff': exr_diff_collect(max_diff=0, max_case=None),
 'ulp_diff': exr_diff_collect(max_diff=0, max_case=None)}

## Методы округления к `int`, отсутствующие в `math`

In [1]:
_ext_sql_round_05to0 = math.nextafter(0.5, 0)

def exr_sql_round_int(x, domain=math):
    """
    Целая реализация округления до ближайшего большего по модулю, 
    аналогична функции C/C++/SQL/... `round()`, IEEE 754: roundTiesToAway 
    (математическое округление), ROUND_HALF_UP.
    """
    if domain == math:
        return (math.trunc(x + math.copysign(0.5, x)) 
                if abs(x) != _ext_sql_round_05to0 else 0)
    xf, xi = np.modf(x)
    return xi + domain.copysign(abs(xf) >= 0.5, x)  ## TODO np.where или что-то
    #return domain.trunc(x + domain.copysign(0.5, x))
    # Варианты:
    # math.trunc(x + math.copysign(0.5, x)) if abs(x) != math.nextafter(0.5, 0)
    # t = math.trunc(x); t + math.copysign(abs(x - t) >= 0.5, x)
    # xf, xi = math.modf(x); xi + math.copysign((0.5 <= abs(xf)), xf)
    # xf, xi = math.modf(x); xi + ((0.5 <= xf) if 0 <= xf else -(0.5 <= -xf))

def exr_half_down_int(x, domain=math):
    """
    Целая реализация округления до ближайшего меньшего по модулю, 
    ROUND_HALF_DOWN.
    """
    if domain == math:
        return math.ceil(x - 0.5) if 0 <= x else math.floor(x + 0.5)
    return np.where(0 <= x, np.ceil(x - 0.5), np.floor(x + 0.5))

def exr_up_int(x, domain=math):
    """
    Целая реализация округления к большему по модулю, ROUND_UP.
    """        
    if domain == math:
        return math.ceil(x) if 0 <= x else math.floor(x)
    return np.where(0 <= x, np.ceil(x), np.floor(x))

def exr_05up_int(x, domain=math):
    """
    Целая реализация округления к меньшему по модулю, если последняя 
    цифра результата не получается 0 или 5, в этом случае округление 
    к большему по модулю, ROUND_05UP.
    """
    tx = domain.trunc(x)
    ux = exr_up_int(x, domain=domain)
    return np.where(tx%5, tx, ux)

# Интерфейсы к стандартным методам округления

def exr_ceil_int(x, domain=math):
    return domain.ceil(x)
    # Варианты:
    # x - x%-1.
    # -(x//-1.)
    # math.ceil(x)

def exr_trunc_int(x, domain=math):
    return domain.trunc(x)
    # Варианты:
    # x - math.fmod(x, 1.)
    # math.trunc(x)
    # math.modf(x)[1]
    # x - math.modf(x)[0]

def exr_floor_int(x, domain=math):
    return domain.floor(x)
    # Варианты:
    # x//1.
    # x - x%1.
    # float(math.floor(x))
    # divmod(x, 1.)[0]

def exr_np_round_int(x, domain=np):
    return np.round(x)

_exr_int_sw = {
    decimal.ROUND_CEILING:   exr_ceil_int,
    decimal.ROUND_DOWN:      exr_trunc_int,
    decimal.ROUND_FLOOR:     exr_floor_int,
    decimal.ROUND_HALF_EVEN: exr_np_round_int,
    decimal.ROUND_HALF_UP:   exr_sql_round_int,
    decimal.ROUND_HALF_DOWN: exr_half_down_int,
    decimal.ROUND_UP:        exr_up_int,
    decimal.ROUND_05UP:      exr_05up_int,
}


NameError: name 'math' is not defined

### Спойлер тестов

In [14]:

class TestRoundingsInt(unittest.TestCase):
    def test_math_roundings(self):
        for r, f in _exr_int_sw.items():
            with self.subTest(r=r):
                for x in np.arange(-2., 2., 0.25):
                    self.assertEqual(f(x), exr_decimal_round(x, rounding=r),
                                     msg=f"{x=}")
                    for i in [-math.inf, math.inf]:
                        y = x
                        for _ in range(3):
                            y = math.nextafter(y, i)
                            self.assertEqual(f(y), 
                                             exr_decimal_round(y, rounding=r),
                                             msg=f"{x=} {i=} {_=} {y=}")
    def test_np_scalar_roundings(self):
        for r, f in _exr_int_sw.items():
            with self.subTest(r=r):
                for x in np.arange(-2., 2., 0.25):
                    self.assertEqual(f(x, domain=np), 
                                     exr_decimal_round(x, rounding=r),
                                     msg=f"{x=}")
                    for i in [-math.inf, math.inf]:
                        y = x
                        for _ in range(3):
                            y = np.nextafter(y, i)
                            self.assertEqual(f(y, domain=np), 
                                             exr_decimal_round(y, rounding=r),
                                             msg=f"{x=} {i=} {_=} {y=}")
    def test_np_array_roundings(self):
        for r, f in _exr_int_sw.items():
            with self.subTest(r=r):
                x = np.arange(-2., 2., 0.25)
                rx = np.vectorize(lambda x: exr_decimal_round(x, rounding=r)
                                 )(x)
                np.testing.assert_equal(f(x, domain=np), rx,
                                               err_msg=f"{x=}")
                for i in [-math.inf, math.inf]:
                    y = x
                    for _ in range(3):
                        y = np.nextafter(y, i)
                        ry = np.vectorize(
                                lambda x: exr_decimal_round(x, rounding=r)
                            )(y)
                        np.testing.assert_equal(f(y, domain=np), ry,
                                               err_msg=f"{i=} {y=}")

unittest.main(argv=[''], verbosity=0, exit=False)

----------------------------------------------------------------------
Ran 3 tests in 0.043s

OK


## Машинное и человекочитаемое представление

Python, как и многие языки программирования, определяет, во-первых, 
[представление `float`](https://docs.python.org/3/c-api/float.html) как 
`binary64` на платформах с поддержкой IEEE 754 (IEC 60559), и во-вторых, 
взаимно однозначное соответствие между подмножеством строк и значениями 
`float`: [`float(repr(x)) == x`](https://docs.python.org/3/library/sys.html#sys.float_repr_style).

Для распространённых обобщённых типов: `np.half`, `np.single`, `np.double`, 
`np.longdouble`, `fractions.Fraction`, `decimal.Decimal` это соответствие 
выглядит как: `type(x)(str(x)) == x`.

Но, несмотря на это взаимно однозначное соответствие значений, их 
"естественное" округление немного различается, что порождает сюрпризы вида: 
`5.0 == exr_easy_round(5.00001, 5, 'ROUND_DOWN', fmt_out=True)` - "Как так? 
Вроде как, идеальное усечение до пяти знаков, уничтожает 1 в пятом знаке?!".

Таким образом, строго говоря, во фразе: "Округление/усечение до определённой 
цифры", семантика позиции цифры ни черта не определённая. Строго говоря, 
требуется явное указание, что это имеется ввиду позиция цифры для машинного 
представления. Хотя неформально, общепринято опускать это уточнения.

Но иногда, может потребоваться иное. Вобщем, задачи округления `x` и 
`str(x)`, немного различаются, обе могут иметь некоторый смысл и имеют 
достаточно строгую постановку.

Несложно показать, что задача округления `str(x)` для округления к целому 
идентична задаче округления `x` (и для `ndigits < 0` тоже экивалентна, но
учёт ошибок округления отличается). А для `ndigits > 0` имеет решение вида:

$$round\_int\left(\left( x \pm \frac{ULP(x)}{2} \right) \otimes 10^{ndigits}
                  \right) \oslash 10^{ndigits}$$

## Требования к точности вычислений при округлении/усечении



In [67]:
x = np.half('2.0115')
n = 5
r = np.round(x, n)
print(f"np.round: {type(r), r=}")
t2 = type(x)(2)
i2, f2 = np.divmod(x, t2)
r = i2*t2 + np.round(f2, n)
print(f"np.divmod: {type(r), r=}")
rc = exr_decimal_round(x, n, 'ROUND_HALF_EVEN', fmt_out='True')
print(f"exr_decimal_round: {type(rc), rc=}")

np.round: type(r), r=(<class 'numpy.float16'>, inf)
np.divmod: type(r), r=(<class 'numpy.float16'>, 2.012)
exr_decimal_round: type(rc), rc=(<class 'numpy.float16'>, 2.012)


In [83]:
x = np.half('215')
x = np.nextafter(np.half('215'), np.half(0))
n = -1
r = np.round(x, n)
print(f"np.round: {type(r), r=}")
# t2 = type(x)(2)
# i2, f2 = np.divmod(x, t2)
# r = i2*t2 + np.round(f2, n)
# print(f"np.divmod: {type(r), r=}")
rc = exr_decimal_round(x, n, 'ROUND_HALF_EVEN', fmt_out='True')
print(f"exr_decimal_round: {type(rc), rc=}")

np.round: type(r), r=(<class 'numpy.float16'>, 210.0)
exr_decimal_round: type(rc), rc=(<class 'numpy.float16'>, 210.0)


In [86]:
print(np.finfo(x))

Machine parameters for float16
---------------------------------------------------------------
precision =   3   resolution = 1.00040e-03
machep =    -10   eps =        9.76562e-04
negep =     -11   epsneg =     4.88281e-04
minexp =    -14   tiny =       6.10352e-05
maxexp =     16   max =        6.55040e+04
nexp =        5   min =        -max
smallest_normal = 6.10352e-05   smallest_subnormal = 5.96046e-08
---------------------------------------------------------------



In [105]:
x = np.half('2045')
print(np.nextafter(x, type(x)(-np.inf)), x, np.nextafter(x, type(x)(np.inf)))
print(np.round(x), type(x)(exr_sql_round_int(x)))
print(np.floor(x), np.trunc(x), np.ceil(x))

2044.0 2045.0 2046.0
2045.0 2045.0
2045.0 2045.0 2045.0


In [160]:
n = 0
q = np.half(10**-n)
lim = type(q)(4)
qinf = type(q)(np.inf)
x = type(q)(0.05)
xmax = type(q)(np.inf)
print(f"{x, xmax=}")
cnt = 1
while x < qinf:
    r = np.round(x, n)
    rc = exr_decimal_round(x, n, 'ROUND_HALF_EVEN', fmt_out='True')
    if r != rc:
        print(f"{x, n=}, {r, rc=}")
        break
    x = np.nextafter(x, qinf)
    cnt += 1
cnt

x, xmax=(0.05, inf)


/var/folders/wy/z0gbkfgs7mv24ryqkdm90rm40009rh/T/ipykernel_12065/3978451147.py:15: RuntimeWarning: overflow encountered in nextafter
  x = np.nextafter(x, qinf)


20891

In [156]:
r, rc, exr_decimal_round(x, n, 'ROUND_HALF_EVEN', fmt_out='False')

(6400.0, 6500.0, 6500.0)

In [157]:
exr_decimal_round(x, n, 'ROUND_HALF_EVEN', fmt_out='True')

6500.0

In [158]:
x10=x/q
np.nextafter(x10, -qinf), x10, np.nextafter(x10, qinf)

(64.44, 64.5, 64.56)

In [154]:
decimal.Decimal(float(x10))

Decimal('256.5')

(2566.0, '2566.0')

В случае $ndigits \lt 0$

0.01172

### Вычисление на точности входного аргумента

В случае $ndigits = 0$ выражение `type(x)(round_int(x))` всегда точно, 
поскольку, в худшем случае, минимального $ULP(round\_int(x)) = 1$ ближайшее 
нецелое $x_{err} = x - 0.5$.

В случае $ndigits \gt 0$, если вычислять 
$round\_int(x \otimes 10^{ndigits}) \oslash 10^{ndigits}$ на точности 
входного аргумента, как это делает функция Numpy [`np.round(a, decimals=0, out=None)`](https://numpy.org/doc/stable/reference/generated/numpy.round.html)
то результат округления/усечения будет неточен. Пример 
из документации `np.round(56294995342131.5, 3) == 56294995342131.51` немного 
искусственный, т.к. округляют до трёх знаков, число с 
наименьшим значащим разрядом `0.0078125`, но тем не менее, 
поскольку стандартный `round(56294995342131.5, 3) == 56294995342131.5`, то 
можно же получить корректный ответ. Для небольших чисел, 
к примеру, в выражении `round(16.055*10**2)/10**2`, иногда ошибки округления 
приводят к получению вместо нормального округления машинного представления,
для которого дано выражение, округления человекочитаемого представления.

Более неприятные грабли могут возникать при неточном усечении до заданного 
знака, если предыдущие случаи, типа, неточное округление - ну, бывает. Но, 
к примеру, когда при `x=67108863.949999996`, `math.trunc(x*100)/100 > x`, 
то это может быть немного более неприятно.

Зато шустро, наверное, при реализации усечений (ceil, trunc, floor или 
ROUND_UP) стоит добавить min()/max().

Точность вычисления: $10^{-ndigits}$ для $ndigits \gt 0$ и 
$0.5 \times ULP(x)$ для $ndigits \le 0$.

In [13]:
for x, n in [(56294995342131.5, 3), (16.055, 2)]:
    print(f"{x, n=}")
    print(f"{round(x, n)=}")
    print(f"{np.round(x, n)=}, {np.round(x, n) == round(x, n)=}")
    print(f"{round(x*10**n)/10**n=}, {round(x*10**n)/10**n == round(x, n)=}")

print(f"{round(x*10**n)/10**n == exr_decimal_round(x, n, 'ROUND_HALF_EVEN', fmt_out=True) =}")
print(f"{round(x*10**n)/10**n == exr_decimal_round(x, n, 'ROUND_HALF_EVEN', fmt_out=False) =}")
print("---")
x, n = 67108863.949999996, 2
print(f"{x, n=}")
print(f"{math.trunc(x*100)/100=} {math.trunc(x*100)/100 <= x =}")


x, n=(56294995342131.5, 3)
round(x, n)=56294995342131.5
np.round(x, n)=np.float64(56294995342131.51), np.round(x, n) == round(x, n)=np.False_
round(x*10**n)/10**n=56294995342131.51, round(x*10**n)/10**n == round(x, n)=False
x, n=(16.055, 2)
round(x, n)=16.05
np.round(x, n)=np.float64(16.06), np.round(x, n) == round(x, n)=np.False_
round(x*10**n)/10**n=16.06, round(x*10**n)/10**n == round(x, n)=False
round(x*10**n)/10**n == exr_decimal_round(x, n, 'ROUND_HALF_EVEN', fmt_out=True) =False
round(x*10**n)/10**n == exr_decimal_round(x, n, 'ROUND_HALF_EVEN', fmt_out=False) =True
---
x, n=(67108863.949999996, 2)
math.trunc(x*100)/100=67108863.95 math.trunc(x*100)/100 <= x =False


### Вычисление путём преобразования в строку

Для случая, неограниченного значения `ndigits` и произвольного значения `x`.

Метод Python `float.__round__(self, ndigits=None, /)` как раз именно так и 
реализован, через функцию `_Py_dg_dtoa()`, что эквивалентно 
`f"{x:.{ndigits}f}"`. Возможно, это наиболее простой, если не единственный 
способ гарантировать соответствие функции округления и форматного 
преобразования в жизненном цикле разработки Python. Худший случай - 310 
символов, конечно, немного удручает, но зато сильные гарантии корректности 
и совместимости.

При реализации иных методов округления этим способом, представляется три 
потенциально возможные пути использования стандартного алгоритма форматного 
преобразования:

- Использовать преобразование типа `f"{x:.1074f}"`, а округлять эту строку
  уже самостоятельно, по требуемым правилам. Собственно, где 310,
  там и 1384, т.к. в худшем случае будет 309 знаков целой части и 1074 `0` в
  дробной. Кроме того, возможно, 1384 можно превратить в функцию от ndigits;
- Использовать преобразование `f"{x:.768g}"`;
- Изменить метод округления процессора на время вызова форматного
  преобразования.

Впрочем, на фоне, форматного преобразования, и 
`exr_easy_round()`/`exr_decimal_round()` не так уж плохо выглядят. С 
гарантиями соответствия форматным преобразованиям по-хуже, зато гарантии 
корректности методов округления по-лучше.

[1] Guy L. Steele, Jr. and Jon L. White, "How to Print Floating-Point 
    Numbers Accurately", [Proc. ACM SIGPLAN '90, pp. 112-126];
    
[2] David M. Gay, "Correctly Rounded Binary-Decimal and Decimal-Binary 
    Conversions", 1990


In [61]:
for t in [float, np.half, np.single, np.double, np.longdouble]:
    bs = [np.finfo(t).smallest_normal, np.finfo(t).smallest_subnormal, 
          np.finfo(t).max, t(0.5)]
    xs = [t(np.nextafter(b, d)) for b in bs
          for d in ([np.inf, b, 0] if abs(b) < np.finfo(b).max else
                    [b, 0])]
    cvt = exr_decimal_from_longdouble if t == np.longdouble else float
    lens = [len(decimal.Decimal(cvt(x)).as_tuple().digits)
            for x in xs]
    print(f"{xs[0].__class__.__name__:10} {max(lens):6d} {lens=}")

float         767 lens=[767, 715, 767, 750, 751, 1, 309, 309, 53, 1, 54]
float16        20 lens=[20, 10, 20, 17, 17, 1, 5, 5, 11, 1, 12]
float32       112 lens=[112, 89, 112, 104, 105, 1, 39, 39, 24, 1, 25]
float64       767 lens=[767, 715, 767, 750, 751, 1, 309, 309, 53, 1, 54]
longdouble  11514 lens=[11514, 11451, 11514, 11494, 11495, 1, 4933, 4933, 64, 1, 65]


In [62]:
# Реализация float.__round__(self, ndigits=None, /)
# cpython/Objects/floatobject.c:double_round
# При отсутствии ndigits использует round(x) или round(x/2.)*2.
# При наличии ndigits использует _Py_dg_dtoa(x, 3, ndigits, ...),
# Что соответствует: PyOS_double_to_string(val=x, format_code='f', precision=ndigits, ...)
# x.__format__(f'.{ndigits}f')
if 0:
    python_compare = [("True", "x.__round__(n)"),
                      ("fn is not None", "x.__format__(fn)")]
    exr_compare = [(cnd, f"{f}(x, n, fmt_out={fo})")
                   for cnd, f in [("n is not None", "exr_easy_round"), 
                                  ("True", "exr_decimal_round")]
                   for fo in [None, True]]
    compare = python_compare  + exr_compare
    r = 4
    for n in [None, 0, 3, -3]:
        fn = f'.0f' if n is None else None if n < 0 else f'.{n}f'
        for x in [2.0115, 2.0115e150, 2.0115e300, 2.0115e-300]:
            print(f"{x, n, fn=}")
            maxstnt = max([len(stmt) for cnd, stmt in compare])
            for cnd, stmt in compare:
                print(f"{stmt :{maxstnt}} ", end="")
                if eval(cnd):
                    get_ipython().run_line_magic("timeit", f"-r {r} {stmt}")
                else:
                    print("Недоступно")
print("Хорь")

Хорь


### Вычисление с использванием `fma()`

Возможно этот вариант можно назвать вычислением с удвоенной точностью в 
технике 'double-double', но в нём, для типа Python `float`, используется 
только одна базовая операция. 

Вариант применим в предположении, что $10^{ndigits}$ имеет точное 
представление и что умножение на него не вызовет переполнения 
($\left| ndigits \right| \le mant\_dig \times log_{2}{5} \ \wedge \ \left| x \right| \le \frac{max}{10^{ndigits}}$).

#### `int` неограниченной разрядности и `math.fma()`

In [63]:
round(-0., 0)

-0.0

In [64]:
exr_easy_round(-0., 0), exr_decimal_round(-0., 0)

(-0.0, -0.0)

In [65]:
x = 0.
-0.0 == 0.0, -x

(True, -0.0)

In [285]:
def _exr_nextafters(x, steps):
    if np.isnan(x):
        return [x]
    t = type(x)
    r = [ x ]
    for _ in range(steps):
        n = t(np.nextafter(r[0], -np.inf))
        if n != r[0]:
            r.insert(0, n)
        n = t(np.nextafter(r[-1], np.inf))
        if n != r[-1]:
            r.append(n)
    if x:
        r += [-y for y in r]
    else:
        r += [-x]
    return r

def _exr_defaults(a, dflt):
    if a is None:
        return dflt  #if hasattr(a, '__iter__') else [dflt]
    elif hasattr(a, '__iter__') and not isinstance(a, str):  # TODO
        return a
    return [a]

def _exr_call(f, *args, **kwargs):
    return f(*args, **{arg: val for arg, val in kwargs.items() if val is not None})

class _exr_diff_collect:
    def __init__(self):
        self.max_diff = 0
        self.max_case = None
    def add(self, q, std, chk, x, args):
        if not np.isfinite(q):
            print(f"exr_diff_collect.add({q=}, {std=}, {chk=}, {x=}, {args=})")
        diff = q*abs(std - chk)
        if diff > self.max_diff:
            self.max_diff = diff
            self.max_case = dict(std=std, chk=chk, x=x, args=args)
    def __repr__(self):
        return f"exr_diff_collect(max_diff={self.max_diff}, max_case={self.max_case})"

_exr_numbers_from_text = [
    '2.675', '2.0115', '20115', '4.00001', '2013.0', "0.222222222222222225",
    "2.71828182845904523536028747135266e25", '56294995342131.5', 
    '16.055', '16.085', 
    # Умножение?
    '8589934591.059999', '2147483647.8799999', '536870911.96999997',
    '268435455.76999998', '67108863.949999996',
    # Деление
    '524287.9999291525', '524287.9999291524', 
    '262143.99998263217', '262143.9999826321',
    '2.654', '1.9', '0.21', '2.01', '201.0', '0.55555555555555555555555', 
    "0.15", "0.145", "0.0000005", 
    '249.99999999999997', '5.465', '0.00261', '0.00161', str(0.15+0.005*62),
    '2.0125', '0.05', '0.25', '0.005', '0.015', '0.025', '0.0115'
]
_exr_numbers_from_text.sort()
for x, y in zip(_exr_numbers_from_text[:-1], _exr_numbers_from_text[1:]):
    assert x < y, f"Duplicate {x, y=}"
_exr_numbers_from_text += [float(x) for x in _exr_numbers_from_text]

def exr_check_floats(f, types_x=None, test_xs=None, test_ndigits=None, 
                     test_roundings=None, test_fmt_outs=None, 
                     nsteps=16, zero_diff=True, inf_nan=True, 
                     verbose=0,
                     standard=exr_decimal_round):
    ndigits_none_exceptions = (OverflowError, ValueError)
    idn = True
    cnt = {'total': 0, 'except': 0, 'diff': 0, 'type': 0, 'zdiff': 0}
    abs_diff = _exr_diff_collect()
    ndigit_diff = _exr_diff_collect()
    ulp_diff = _exr_diff_collect()

    def fail_verbose(label, diff):
        nonlocal idn, verbose, chk, std, x, chk_args
        if diff:
            idn = False
            cnt[label] += 1
            if verbose:
                print(f"{label}: {chk, std, x=} {chk_args=}")
                verbose -= 1
            
    for t in _exr_defaults(types_x, _exr_float_types):
        max_spacing = np.finfo(t).max
        min_inversible = 1/np.finfo(t).max
        for x0 in _exr_defaults(test_xs, 
                                (list(np.arange(0., 4.01, 1/16))
                                 + [str(a) + str(b) + k*'0' + '.' 
                                    + k*'0' + str(a) + str(b)
                                    for a in range(10)
                                    for b in range(0, 10, 5)
                                    for k in range(3)]
                                 + ([np.inf, np.nan] if inf_nan else [])
                                 #+ _exr_numbers_from_text
                                )):
            for x in _exr_nextafters(t(x0), nsteps):
                for ndigits in _exr_defaults(test_ndigits, 
                                             [None] + list(range(-3, 4, 1))):
                    for rounding in _exr_defaults(test_roundings, 
                                                  _exr_roundings):
                        for fmt_out in _exr_defaults(test_fmt_outs,
                                                     [None, True, False]):
                            try:
                                std = standard(x, ndigits=ndigits, 
                                               rounding=rounding, 
                                               fmt_out=fmt_out)
                            except ndigits_none_exceptions as e:
                                std = e
                                if ndigits is not None  or  np.isfinite(x):
                                    assert False, (
                                        f"{std, x=} {ndigits, rounding, fmt_out, standard=}"
                                    )
                            chk_args = {
                                'ndigits': ndigits
                            }
                            if not isinstance(test_roundings, str):
                                chk_args['rounding'] = rounding
                            if (test_fmt_outs is None or 
                                hasattr(test_fmt_outs, '__iter__')):
                                chk_args['fmt_out'] = fmt_out
                            try:
                                chk = _exr_call(f, x, **chk_args)
                            except ndigits_none_exceptions as e:
                                chk = e
                                if ndigits is not None  or  np.isfinite(x):
                                    fail_verbose("except", True)
                                continue
                            cnt["total"] += 1
                            if type(chk) != type(std):
                                fail_verbose("type", True)
                            if (chk or std):
                                diff = (not (chk == std) and
                                        not (isinstance(chk, _exr_float_types) and
                                             np.isnan(chk) and np.isnan(std)))
                                fail_verbose("diff", diff)
                            else:
                                diff = not ((chk == std) and 
                                            (str(chk) == str(std)))
                                fail_verbose("zdiff", diff)
                            if diff:
                                idn = False
                                abs_diff.add(1, std, chk, x, chk_args)
                                ndigit_diff.add(10**(ndigits if ndigits else 0), 
                                                std, chk, x, chk_args)
                                if (abs(x) < max_spacing and
                                    abs(np.spacing(x)) > min_inversible):
                                    ulp_diff.add(t(1.)/np.spacing(x), 
                                                 std, chk, x, chk_args)
    return dict(idn=idn, cnt=cnt, 
                abs_diff=abs_diff, ndigit_diff=ndigit_diff, ulp_diff=ulp_diff)

exr_check_floats(round, test_roundings=decimal.ROUND_HALF_EVEN, 
                 test_fmt_outs=True, inf_nan=False,
                 types_x=np.single, #np.single, #float,
                 verbose=10)
# exr_check_floats(round, test_roundings=decimal.ROUND_HALF_EVEN, test_fmt_outs=True)

diff: chk, std, x=(np.float32(5.0), np.float32(5.1), np.float32(5.05)) chk_args={'ndigits': 1}
diff: chk, std, x=(np.float32(-5.0), np.float32(-5.1), np.float32(-5.05)) chk_args={'ndigits': 1}
diff: chk, std, x=(np.float32(50.0), np.float32(50.01), np.float32(50.005)) chk_args={'ndigits': 2}
diff: chk, std, x=(np.float32(-50.0), np.float32(-50.01), np.float32(-50.005)) chk_args={'ndigits': 2}
diff: chk, std, x=(np.float32(15.2), np.float32(15.1), np.float32(15.15)) chk_args={'ndigits': 1}
diff: chk, std, x=(np.float32(-15.2), np.float32(-15.1), np.float32(-15.15)) chk_args={'ndigits': 1}
diff: chk, std, x=(np.float32(150.02), np.float32(150.01), np.float32(150.015)) chk_args={'ndigits': 2}
diff: chk, std, x=(np.float32(-150.02), np.float32(-150.01), np.float32(-150.015)) chk_args={'ndigits': 2}
diff: chk, std, x=(np.float32(1500.002), np.float32(1500.001), np.float32(1500.0015)) chk_args={'ndigits': 3}
diff: chk, std, x=(np.float32(-1500.002), np.float32(-1500.001), np.float32(-1500.00

{'idn': False,
 'cnt': {'total': 64976, 'except': 0, 'diff': 216, 'type': 0, 'zdiff': 0},
 'abs_diff': exr_diff_collect(max_diff=0.100006103515625, max_case={'std': np.float32(500.1), 'chk': np.float32(500.0), 'x': np.float32(500.05002), 'args': {'ndigits': 1}}),
 'ndigit_diff': exr_diff_collect(max_diff=1.46484375, max_case={'std': np.float32(4500.011), 'chk': np.float32(4500.01), 'x': np.float32(4500.0107), 'args': {'ndigits': 3}}),
 'ulp_diff': exr_diff_collect(max_diff=209715.0, max_case={'std': np.float32(5.1), 'chk': np.float32(5.0), 'x': np.float32(5.05), 'args': {'ndigits': 1}})}

In [278]:
np.float32(19000.041) - np.float32(19000.043)

np.float32(-0.001953125)

In [279]:
(np.float32(19000.041) - np.float32(19000.043))/0.001

np.float32(-1.9531249)

In [280]:
(np.float32(19000.041) - np.float32(19000.043))/np.spacing(np.float32(19000.041))

np.float32(-1.0)

In [254]:
r = exr_decimal_round(x, -3, fmt_out=True, rounding=decimal.ROUND_HALF_EVEN)
x, r

/var/folders/wy/z0gbkfgs7mv24ryqkdm90rm40009rh/T/ipykernel_22131/2716139783.py:27: RuntimeWarning: overflow encountered in cast
  return x_to(dx.scaleb(ndigits).to_integral_value(rounding=rounding).


(np.float16(65500.0), np.float16(inf))

In [255]:
round(x, -3)

/var/folders/wy/z0gbkfgs7mv24ryqkdm90rm40009rh/T/ipykernel_22131/4148338072.py:1: RuntimeWarning: overflow encountered in multiply
  round(x, -3)


np.float16(inf)

In [220]:
t = np.longdouble
x = np.nextafter(t(1), 0)
np.spacing(x)

/var/folders/wy/z0gbkfgs7mv24ryqkdm90rm40009rh/T/ipykernel_22131/4124263464.py:3: RuntimeWarning: invalid value encountered in spacing
  np.spacing(x)


np.longdouble('nan')

In [163]:
exr_decimal_round(np.float16(0.1245), 3, fmt_out=True, rounding=decimal.ROUND_HALF_EVEN)

np.float16(0.125)

In [179]:
x = np.longdouble(123050.005)
x.__round__(2)

np.longdouble('123050.01')

In [166]:
x = np.float16(0.1245)
x.as_integer_ratio(), float(x).as_integer_ratio()

((255, 2048), (255, 2048))

In [165]:
255/2048

0.12451171875

In [168]:
round(0.1245, 3), round(np.float16(0.1245), 3)

(0.124, np.float16(0.124))

In [154]:
l = ['123' + str(a) + str(b) + k*'0' + '.' + k*'0' + str(a) + str(b)
 for a in range(10) 
 for b in range(0, 10, 5)
 for k in range(3)]
len(l), l

(60,
 ['12300.00',
  '123000.000',
  '1230000.0000',
  '12305.05',
  '123050.005',
  '1230500.0005',
  '12310.10',
  '123100.010',
  '1231000.0010',
  '12315.15',
  '123150.015',
  '1231500.0015',
  '12320.20',
  '123200.020',
  '1232000.0020',
  '12325.25',
  '123250.025',
  '1232500.0025',
  '12330.30',
  '123300.030',
  '1233000.0030',
  '12335.35',
  '123350.035',
  '1233500.0035',
  '12340.40',
  '123400.040',
  '1234000.0040',
  '12345.45',
  '123450.045',
  '1234500.0045',
  '12350.50',
  '123500.050',
  '1235000.0050',
  '12355.55',
  '123550.055',
  '1235500.0055',
  '12360.60',
  '123600.060',
  '1236000.0060',
  '12365.65',
  '123650.065',
  '1236500.0065',
  '12370.70',
  '123700.070',
  '1237000.0070',
  '12375.75',
  '123750.075',
  '1237500.0075',
  '12380.80',
  '123800.080',
  '1238000.0080',
  '12385.85',
  '123850.085',
  '1238500.0085',
  '12390.90',
  '123900.090',
  '1239000.0090',
  '12395.95',
  '123950.095',
  '1239500.0095'])

In [75]:
isinstance(np.single(1), tuple(_exr_float_types))

True

In [53]:
exr_decimal_round(math.inf, fmt_out=True)

TypeError: '<=' not supported between instances of 'NoneType' and 'int'

In [35]:
exr_decimal_round(math.nextafter(math.inf, 0, steps=3), fmt_out=True)

179769313486231530000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [36]:
exr_decimal_from_longdouble(math.nextafter(math.inf, 0, steps=3))

Decimal('179769313486231530897721233037308123670616306789073675249892268094006430729083690016351721253485562472381149100800360677385384443039559303319707778831664302202613944879922875676132031068675798593321439621195390289237448346024743081549599456914715570497692457243261573026320039665798467034080499419303926300672')

In [24]:
def nzround(number, ndigits=None):
    return 0 + round(number, ndigits=ndigits)
exr_check_floats(nzround, test_roundings=decimal.ROUND_HALF_EVEN, test_fmt_outs=False, 
                 types_x=float, verbose=10)

zdiff: chk, std, x=(0.0, -0.0, -8e-323) chk_args={'ndigits': -3}
zdiff: chk, std, x=(0.0, -0.0, -8e-323) chk_args={'ndigits': -2}
zdiff: chk, std, x=(0.0, -0.0, -8e-323) chk_args={'ndigits': -1}
zdiff: chk, std, x=(0.0, -0.0, -8e-323) chk_args={'ndigits': 0}
zdiff: chk, std, x=(0.0, -0.0, -8e-323) chk_args={'ndigits': 1}
zdiff: chk, std, x=(0.0, -0.0, -8e-323) chk_args={'ndigits': 2}
zdiff: chk, std, x=(0.0, -0.0, -8e-323) chk_args={'ndigits': 3}
zdiff: chk, std, x=(0.0, -0.0, -7.4e-323) chk_args={'ndigits': -3}
zdiff: chk, std, x=(0.0, -0.0, -7.4e-323) chk_args={'ndigits': -2}
zdiff: chk, std, x=(0.0, -0.0, -7.4e-323) chk_args={'ndigits': -1}
cnt=34064 idn=False cnt_diff=0 cnt_zdiff=6703
abs_diff=exr_diff_collect(max_diff=0, max_case=None)
ndigit_diff=exr_diff_collect(max_diff=0, max_case=None)
ulp_diff=exr_diff_collect(max_diff=0, max_case=None)


In [20]:
(-0. + 0., -0. + (-0.)), (0. + 0., 0. + (-0.))

((0.0, -0.0), (0.0, 0.0))

In [ ]:
class exr_diff_collect:
    def __init__(self):
        self.max_diff = 0
        self.max_case = None
    def check(self, std, chk, x, args):
        diff = abs(std - chk)
        if diff > self.max_diff:
            self.max_case = dict(std=std, chk=chk, x=x, args=args)

In [330]:
exr_defaults(None, _exr_float_types)

[float, numpy.float16, numpy.float32, numpy.float64, [...]]

In [293]:
def test(**kwargs):
    print(f"{kwargs=}")

def exr_call(f, **kwargs):
    kwa = dict()
    for arg, val in kwargs.items():
        if val is not None:
            kwa[arg] = val
    return f(**kwa)

def exr_call(f, **kwargs):
    kwa = {arg: val for arg, val in kwargs.items() if val is not None}
    return f(**kwa)

def exr_call(f, **kwargs):
    return f(**{arg: val for arg, val in kwargs.items() if val is not None})

exr_call(test, a=None, b=1)

kwargs={'b': 1}


In [249]:
type 

Signature: sum(iterable, /, start=0)
Docstring:
Return the sum of a 'start' value (default: 0) plus an iterable of numbers

When the iterable is empty, return the start value.
This function is intended specifically for use with numeric values and may
reject non-numeric types.
Type:      builtin_function_or_method

In [245]:
def sql_round_python(x: 'Uninon[float, np.double]', ndigits):
    assert ndigits >= 0, "Не реализовано"
    q = 10**ndigits
    xq = x*q
    rxq = exr_sql_round_int(xq)
    if xq%0.5 == 0.:  # варианты xq.is_integer(), xq == rxq, для усечений
        xql = math.fma(x, q, -xq)
        print(f"{xql=}")
        rxq += exr_sql_round_int(xql)
    return rxq/q

x

xql=0.0
xql=0.0


(6294995342131.5, -6294995342131.5)

Возможно этот вариант можно назвать вычислением с удвоенной точностью в 
технике 'double-double', но в нём, для типа Python `float`, используется 
только одна базовая операция. 

Вариант применим в предположении, что $10^{ndigits}$ имеет точное 
представление и что умножение на него не вызовет переполнения 
($\left| ndigits \right| \le mant\_dig \times log_{2}{5} \ \wedge \ \left| x \right| \le \frac{max}{10^{ndigits}}$).

```
q = 10**ndigits
xq = x
rxq = round_int(xq)
if xq%0.5 == 0.:  # варианты xq.is_integer(), xq == rxq, для усечений
    xql = math.fma(x, q, -xq)
    rqx += round_int_low(xq, xql)
return rxq/q
```

Поскольку `abs(xql) <= 0.5*math.ulp(xq)`, `xql` влияет на результат лишь 
в некоторых случаях. Ещё пара моментов. Во-первых, этот код, в операции 
сложения использует неограниченную разрядность `int`. Во-вторых, в операции 
деления использует тот факт, что `rxq.__truediv__(q)` имеет ошибку не более 
половины последнего разряда (т.е. получает точное значениие, если оно 
представимо).

В этих случаях для типов Numpy требуется, либо выполнить деление с 
точностью `0.5*ULP(x)`:

```
q = numpy_type(10**ndigits)
xq = x
rxq = round_int(xq)
rxqq = rxq/q
adxq = np.abs(xq - rxq)
need_xql = np.where((adxq == 0.5) | (adxq == 0.))  # np.where(xq == rxq), для усечений
if len(need_xql[0]):
    xql = low_mul_or_fma(x[need_xql], q, xq[need_xql])
    rqxl = round_int_low(x[need_xql], xql)
    rxq0 = rxqq[need_xql]*q
    rxql0 = low_mul_or_fma(rxqq[need_xql], q, rxq0)
    s = rxq[need_xql] - rxq0
    e = (rxq[need_xql] - s) - rxq0  # Нужен ли? Член порядка ULP(ULP(x))
    e += rqxl
    e -= rxql0
    rxqq[need_xql] += (s + e)/q
return rxqq
```

Либо удовлетворится точностью `1.0*ULP(x)`, возможно, с коррекцией для усечений:
```
q = numpy_type(10**ndigits)
xq = x
rxq = round_int(xq)
adxq = np.abs(xq - rxq)
need_xql = np.where((adxq == 0.5) | (adxq == 0.))  # np.where(xq == rxq), для усечений
if len(need_xql[0]):
    xql = low_mul_or_fma(x[need_xql], q, xq[need_xql])
    rxql = round_int_low(x[need_xql], xql)
    if no_correction:
        rxq[need_xql] += rxql
    else:
        s = rxq[need_xql] + rxql
        e = rxql - (s - rxq[need_xql])
        need_correct = np.where(e < 0)  # TODO correction
        s[need_correct] = np.nextafter(s[need_correct], np.inf)  # TODO correction
        rxq[need_xql] = s
return rxq/q
```



In [233]:
#x, n = 67108863.949999996, 2
x, n=(56294995342131.5, 3)
q = 10**n
print(f"{x, n, q=}")
print(f"{math.trunc(x*100)/100=} {math.trunc(x*100)/100 <= x =}")
xq = x*q
rxq = math.trunc(xq)
print(f"{rxq=} {rxq == xq =}")
xql = math.fma(x, q, -xq)
print(f"{xql=}")
rxq_p = rxq + math.floor(xql) if 0 <= xq else math.ceil(xql)
print(f"{rxq_p=}")
print(f"{rxq_p/q=} {rxq_p/q <= x =}")
print("---")

q = float(10**n)
xq = x*q
rxq_n0 = float(math.trunc(xq))
print(f"{rxq_n0=} {rxq_n0 == xq =}")
rxq_n0 += float(math.floor(xql) if 0 <= xq else math.ceil(xql))
print(f"{rxq_n0/q=} {rxq_n0/q <= x =}")
print("---")
rxq_n1 = float(rxq)
rxql_n1 = float(math.floor(xql) if 0 <= xq else math.ceil(xql))
print(f"{rxq_n1, rxql_n1=}")
t = rxq_n1
rxq_n1 = rxq_n1 + rxql_n1
rxql_n1 = rxql_n1 - (rxq_n1 - t)
print(f"{rxq_n1, rxql_n1=}")
if rxql_n1 < 0.:
    rxq_n1 = math.nextafter(rxq_n1, 0.)
print(f"{rxq_n1, rxql_n1=}")
print(f"{rxq_n1/q=} {rxq_n1/q <= x =}")
print("---")
rxq_n2 = float(rxq)
rxql_n2 = float(math.floor(xql) if 0 <= xq else math.ceil(xql))
print(f"{rxq_n2, rxql_n2=}")
rxqq_n2 = rxq_n2/q
print(f"{rxqq_n2=}")
rxq0 = rxqq_n2*q
rxql0 = math.fma(rxqq_n2, q, -rxq0)
s = rxq_n2 - rxq0
e = (rxq_n2 - s) - rxq0 
print(f"a {s, e=}")
e -= rxql0
print(f"{s, e=}")
e += rxql_n2
print(f"{s, e=}")
rxqq_n2 += (s + e)/q
print(f"{rxqq_n2=} {rxqq_n2 <= x =}")

x, n, q=(56294995342131.5, 3, 1000)
math.trunc(x*100)/100=56294995342131.5 math.trunc(x*100)/100 <= x =True
rxq=56294995342131504 rxq == xq =True
xql=-4.0
rxq_p=56294995342131500
rxq_p/q=56294995342131.5 rxq_p/q <= x =True
---
rxq_n0=5.62949953421315e+16 rxq_n0 == xq =True
rxq_n0/q=56294995342131.51 rxq_n0/q <= x =False
---
rxq_n1, rxql_n1=(5.62949953421315e+16, -4.0)
rxq_n1, rxql_n1=(5.62949953421315e+16, -4.0)
rxq_n1, rxql_n1=(5.6294995342131496e+16, -4.0)
rxq_n1/q=56294995342131.49 rxq_n1/q <= x =True
---
rxq_n2, rxql_n2=(5.62949953421315e+16, -4.0)
rxqq_n2=56294995342131.51
a s, e=(0.0, 0.0)
s, e=(0.0, -3.8125)
s, e=(0.0, -7.8125)
rxqq_n2=56294995342131.5 rxqq_n2 <= x =True


In [228]:
def quick_two_diff(a, b):
    #assert abs(a) >= abs(b)
    s = a - b
    return s, (a - s) - b

def two_diff(a, b):
    s = a - b
    bb = s - a
    return s, (a - (s - bb)) - (b + bb)

def two_diff1(a, b):
    s = a - b
    return s, (((a - s) - b) if abs(a) >= abs(b) else
               (a - (b + s)))

cnt = 0
maxab = 0 
maxabab = (None, None)
minab = np.inf
minabab = (None, None)

n = 100
ns = 11
rng = math.nextafter(2**10, 0., steps=ns + 1)
for a0 in np.linspace(-rng, rng, n):
    for a in [math.nextafter(a0, (np.inf if s >= 0 else 
                                  -np.inf), steps=abs(s)) 
              for s in range(-ns, ns + 1)]:
        for b0 in np.linspace(-rng, rng, n):
            for b in [math.nextafter(b0, (np.inf if s >= 0 else 
                                          -np.inf), steps=abs(s)) 
                      for s in range(-ns, ns + 1)]:
                t = two_diff(a, b)
                assert t == two_diff1(a, b)
                if True or abs(a) >= abs(b):
                    #assert t == quick_two_diff(a, b)
                    if t != quick_two_diff(a, b):
                        cnt += 1
                        aab = 2*abs(a - b)/abs(a + b)
                        if aab > maxab:
                            maxab = aab
                            maxabab = (a, b)
                        if aab < minab:
                            minab = aab
                            minabab = (a, b)

print(f"{cnt=}")
print(f"{minab, minabab=}")
print(f"{maxab, maxabab=}")

cnt=1002064
minab, minabab=(0.6756756756756733, (506.82828282828285, 1023.9999999999975))
maxab, maxabab=(100.00000000000934, (506.82828282828285, -527.5151515151496))


In [224]:
a, b

(-0.0004780170795795785, 0.0006676598473473468)

In [194]:
math.nextafter?

Signature: math.nextafter(x, y, /, *, steps=None)
Docstring:
Return the floating-point value the given number of steps after x towards y.

If steps is not specified or is None, it defaults to 1.

Raises a TypeError, if x or y is not a double, or if steps is not an integer.
Raises ValueError if steps is negative.
Type:      builtin_function_or_method

In [165]:
t, quick_two_diff(a, b), t == quick_two_diff(a, b), t == quick_two_diff(a, b), a - b

((np.float64(0.0), np.float64(0.0)),
 (np.float64(0.0), np.float64(0.1)),
 False,
 False,
 np.float64(0.0))

In [134]:
a = 3
b = 1
a - b, (-b) - (-a)

(2, 2)

In [86]:
xq = np.linspace(-2., 2., 1001)
rxq = np.trunc(xq)
adxq = np.abs(xq - rxq)
need_xql = np.where((adxq == 0.5) | (adxq == 0.)) # np.where(xq == rxq)
z = xq[need_xql]
rxq[need_xql] = z

In [58]:
xq = np.linspace(-2., 2., 1001)
rxq = np.trunc(xq)
# %timeit xqok = np.where(xq%1. == 0.)
# %timeit xqok1 = np.where(xq == rxq)

%timeit xqok0 = np.where(np.abs(xq - rxq) == 0.5)
%timeit xqok = np.where(xq%0.5 == 0.)
%timeit xqok1 = (xq%0.5 == 0.)
# %timeit xq[xqok] += 1.
# %timeit xq[xqok1] += 1.

3.69 μs ± 154 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
12.6 μs ± 177 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
11.6 μs ± 191 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [76]:
%timeit xf, xi = np.modf(xq); xqok0 = np.where((xf == 0.5) | (xf == 0.))
%timeit xqok = np.where(np.fmod(xq, 1.) == 0.)
%timeit adxq = np.abs(xq - rxq); xqok0 = np.where((adxq == 0.5) | (adxq == 0.))
%timeit dxq = (xq - rxq); xqok1 = np.where((dxq == 0.5) | (dxq == -0.5) | (dxq == 0.))
a_min = np.nextafter(0., 0.5)
a_max = np.nextafter(0.5, 0.)
%timeit adxq = np.abs(xq - rxq); xqok2 = np.where(adxq == np.clip(adxq, a_min=a_min, a_max=a_max))

12.6 μs ± 38.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
11.2 μs ± 35.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
8.65 μs ± 34 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
9.65 μs ± 28 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
13.5 μs ± 46.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [75]:
xf, xi = np.modf(xq)
xf

array([-0.   , -0.996, -0.992, ...,  0.992,  0.996,  0.   ], shape=(1001,))

In [66]:
a_min = np.nextafter(0., 0.5)
a_max = np.nextafter(0.5, 0.)
np.clip(0.5, a_min=a_min, a_max=a_max)

np.float64(0.49999999999999994)

In [40]:
xq = 1.5
rxq = math.trunc(xq)
%timeit xq%0.5 == 0.
%timeit xq.is_integer()
%timeit xq == rxq

36.5 ns ± 1.33 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
24.3 ns ± 1.01 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
27.1 ns ± 0.862 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [43]:
rxq.__truediv__(q)

Signature:      rxq.__truediv__(value, /)
Call signature: rxq.__truediv__(*args, **kwargs)
Type:           method-wrapper
String form:    <method-wrapper '__truediv__' of int object at 0x10fc28a98>
Docstring:      Return self/value.

In [14]:
f"float(10**22):.5f",f"float(10**23):.5fо

('float(10**22):.5f', 'float(10**23):.5f')

In [32]:
for t in [float, np.half, np.single, np.double, np.longdouble]:
    max_ndigits = int(-np.finfo(t).negep/math.log2(5))
    max_abs_x = t(np.finfo(t).max/10**max_ndigits)
    print(f"{max_ndigits:2} {repr(max_abs_x)}")

22 1.7976931348623158e+286
 4 np.float16(6.55)
10 np.float32(3.4028235e+28)
22 np.float64(1.7976931348623158e+286)
27 np.longdouble('1.189731495357231765e+4905')


In [36]:
print(np.finfo(np.half), np.finfo(np.half).smallest_normal/np.finfo(np.half).epsneg)

Machine parameters for float16
---------------------------------------------------------------
precision =   3   resolution = 1.00040e-03
machep =    -10   eps =        9.76562e-04
negep =     -11   epsneg =     4.88281e-04
minexp =    -14   tiny =       6.10352e-05
maxexp =     16   max =        6.55040e+04
nexp =        5   min =        -max
smallest_normal = 6.10352e-05   smallest_subnormal = 5.96046e-08
---------------------------------------------------------------
 0.125


In [26]:
t(1).__class__.__name__

'longdouble'

53/math.log2(5)

In [ ]:
int(10.**11), 10.**11)**2

## Требования к точности вычислений при округлении/усечении

Если вычислять 
$round\_int(x \otimes 10^{ndigits}) \oslash 10^{ndigits}$ на точности 
входного аргумента, как это делает функция Numpy [`np.round(a, decimals=0, out=None)`](https://numpy.org/doc/stable/reference/generated/numpy.round.html)
то результат округления/усечения будет неточен. Пример 
из документации `np.round(56294995342131.5, 3) == 56294995342131.51` немного 
искуственный, т.к. округляют до трёх знаков, число с 
наименьшим значащим разрядом `0.0078125`, но тем не менее, 
поскольку стандартный `round(56294995342131.5, 3) == 56294995342131.5`, то 
можно же получить корректный ответ. Для небольших чисел, 
к примеру, в выражении `round(16.055*10**2)/10**2`, ошибки округления 
приводят к получению округления человекочитаемого представления, хотя 
выражение дано для нормального округления машинного представления.

Более неприятные грабли могут возникать при неточном усечениии до заданного 
знака, если предыдущие случаи, типа, неточное округление - ну, бывает. Но, 
к примеру, когда при `x=67108863.949999996`, `math.trunc(x*100)/100 > x`, 
то это может быть немного более неприятно.

Конечно, если честно использовать наивное округление/усечение вида 
"распечатать в формате с фиксированной запятой и взять нужный кусочек 
от строки", то, в типичном случае, потребуется под сотню десятичных знаков. 
Но, если вычислять согласно определений методов округления, то такая 
высокая точность необязательна.



Для ограничения необходимой точности вычислений разумными значениями 
необходимо ввести ограничения на входные данные:

В выражении $round\_int(x \otimes 10^{ndigits}) \oslash 10^{ndigits}$ 
ошибка деления $\le 0.5 \times ULP(x)$ не влияет на результат, в том смысле, 
что если оба аргумента $round\_int(x \otimes 10^{ndigits})$ и $10^{ndigits}$
точно представимы, то и в результате получится представление точного 
результата в заданом формате. 

Таким образом, необходима точность вычислений, 
которая позволит точно вычислить $x \otimes 10^{ndigits}$ и точно его 
округлить до целого. При естественном ограничении 
`abs(ndigits) <= np.finfo(x).precision` для этого достаточно использовать 
удвоенную точность аргумента.

Варианты точного вычисления $x \otimes 10^{ndigits}$:

1. Python, как большинство современных языков, предоставляет функцию
   `math.fma()` (для Python 3.13 и выше), т.е. можно вычислять примерно так:
   ```
   q = 10**ndigits
   # Точное умножение x*q == xq + xql (abs(xql) <= 0.5*ULP(xq))
   xq = x*q
   xql = math.fma(x, q, -xq)
   # Точное вычисление round_int(xq + xql) == rxq + rxql (abs(rxql) <= 0.5*ULP(rxq))
   rxq, rxql = round_int(xq), 0
   if (rxq - xq)%0.5 == 0:  # или rxq == xq для ceil/trunc/floor/ROUND_UP
       rxq, rxql = round_int_low(rxq, xq, xql)
   # Точное вычисление (rxq, rxql)/q, если оно представимо
   rx = rxq/q
   if not isinstance(rxq, int):
       # Для int, __truediv__() выдаёт точный результат, если он представим
       # А для типов Numpy (np.ndarray или np.longdouble)
       # rxq имеет точность x, поэтому rxql может быть не равен 0
       rxql += math.fma(rx0, q, -rxq)
       rx = (rxq + rxql1)/q
   return rx
   ```
   <br/>

2. Для типов `np.half` и `np.single` более эффективным вариантом будет
   вычисление на типах удвоенной точности или на типе `float`, т.е.
   примерно так:
   ```
   q = 10**ndigits
   type(x)(round_int(ext_type(x)*q)/q)
   ````
   <br/>
   При `ndigits <= 3` и доступности `np.float128` c 64-бит мантиссой, в 
   принципе, при необходимости, возможно использовать вариант 2 для типов 
   `float` и `np.double`
   <br/><br/>

3. Преобразовать аргумент в отношение целых чисел и округлить в рациональной
   арифметике. В случае если `math.fma()` недоступна, для некоторых типов
   и некоторых методов округления, этот вариант может оказаться
   кокурентноспособным по сравнению со старым добрым алгоритмом умножения
   Деккера (double-double без fma())

In [ ]:
# Сравнение деления int, float, np.longdouble
# Реализация int.__truediv__(self, value, /)
# cpython/Objects/longobject.c:long_true_divide
#
sx = '56294995342131500'
sq = '1000'
for t in [int, float, np.longdouble]:
    x = t(sx)
    q = t(sq)
    print(f"{str(x/q)} {t}")
    if t != int:
        x0 = t(np.nextafter(x, 0))
        xi = t(np.nextafter(x, np.inf))
        print(f"\t{str(x0)} {str(x)} {str(xi)}")

In [ ]:
15*53**4

In [ ]:
9.999999999999999e22, 1e23

In [ ]:
for x in np.arange(-2, 2.1, 0.25):
    print(f"{x:5} {x%0.5}")

### А что там у стандартного `round()`

In [ ]:
len("exr_decimal_round(x, n, fmt_out=None)")

In [ ]:
".{}f".format(3)

In [ ]:
# get_ipython().magic(f"timeit -r {r} {cmd}"),
x, n = 11., 22
s = "x.__format__(f'.{n}f')"
x, n = 1.e100, 2
get_ipython().run_line_magic("timeit", s)

In [ ]:
(len(np.finfo(float).max.__format__('.3f')) - 3,
 len(np.finfo(np.longdouble).max.__format__('.3f')) - 3)

In [ ]:
r = '4'
print('x: ', end='')
%timeit -r f"{r}" None

In [ ]:
print?

In [ ]:
for n in range(1, 300):
    a = float("1." + (n-1)*"0" + "1")
    b = float("1." + (n-1)*"0" + "049999")
    # a = float("1." + (n-1)*"0" + "1")
    # b = float("1." + (n-1)*"0" + "150001")    
    if a == b:
        break
    #b = math.nextafter(b, 0)
    ra = round(a, n)
    rb = round(b, n)
    if ra == rb:
        print(f"{n=} {a, b=} {ra, rb=}")
print(f"{n=} {a, b=} {ra, rb=}")


In [ ]:
for n in range(309):
    q = 10**n
    fq = float(q)
    iq = int(q)
    rq = round(q)
    if iq != q:
        print(f"{n=} iq != q {q, fq, iq, rq=}")
    if rq != q:
        print(f"{n=} rq != q {q, fq, iq, rq=}")
    

# Разное

In [ ]:
assert Fale

In [ ]:
# sx = '56294995342131.5'
# n = 3
sx = ''
for u in range(1000):
    x = float(str(u) + sx)
    rx = round(x, n)
    erx = exr_easy_round(x, n, 'ROUND_HALF_EVEN', fmt_out=True)
    erx1 = exr_easy_round(x, n, 'ROUND_HALF_EVEN', fmt_out=False)
    if rx != erx:
        print(f"{x=} {x == erx1 =} : {rx=} {erx=} {erx1=}")
print(f"{x=} {x == erx1 =} : {rx=} {erx=} {erx1=}")

In [ ]:
z = 56294995342131.5/10**9
z, decimal.Decimal(z)

In [ ]:
with decimal.localcontext():  #prec=130):
    for n in range(100):
        x = z
        rx = round(x, n)
        erx = exr_easy_round(x, n, 'ROUND_HALF_EVEN', fmt_out=True)
        erx1 = exr_easy_round(x, n, 'ROUND_HALF_EVEN', fmt_out=False)
        if rx != erx:
            print(f"{n=} {x=} {x == erx1 =} : {rx=} {erx=} {erx1=}")
print(f"{n=} {x=} {x == erx1 =} : {rx=} {erx=} {erx1=}")    

In [ ]:
print(f"{n=} {x=} {x == erx1 =} : {rx=} {erx=} {erx1=}")    

In [ ]:
decimal.Decimal(x).as_tuple().exponent + len(decimal.Decimal(x).as_tuple().digits) + n

In [ ]:
decimal.Decimal(x), decimal.Decimal(f"1e-{n}")

In [ ]:
decimal.Decimal(x).quantize(decimal.Decimal("0." + (n-2)*"0" + "1")) # f"1e-{n-1}"))

In [ ]:
len('95342131494544446468353271484375')

In [ ]:
decimal

In [ ]:
len(decimal.Decimal('1E-23').as_tuple().digits)

In [ ]:
x = 67108863.949999996
q = 100
xq = x*q
xql = math.fma(x, q, -x*q)
txq = math.trunc(xq)
if txq == xq:
    txq += math.floor(xql)
txq, txq/q, txq/q < x

In [ ]:
x = 56294995342131.5
q = 10**3
xq = x*q
xql = math.fma(x, q, -x*q)
print(f"{xq=} {xql=}")
txq = round(xq)
if txq == xq:
    txq += round(xql)
txq, txq/q

In [ ]:
math.ldexp()

In [ ]:
x = 56294995342131.5
q = 10**3
xq = x*q
xql = math.fma(x, q, -x*q)
print(f"{xq=} {xql=}")
txq = round(xq)
if txq == xq:
    txq += round(xql)
txq, txq/q

In [235]:
for x in [16.055, 16.085, -16.055, -16.085]:
    q = 10**2
    xq = x*q
    xql = math.fma(x, q, -x*q)
    print(f"{xq=} {xql=}")
    txq = round(xq)
    if txq == xq or abs(xq - txq) == 0.5:
        print(f"{abs(xq - txq)=} {math.fmod(xq, 2.) + xql=}")
        txq = math.trunc(xq/2)*2 + round(math.fmod(xq, 2.) + xql)
    print(f"{txq=}, {txq/q=} {round(x, 2)=}")

xq=1605.5 xql=-2.842170943040401e-14
abs(xq - txq)=0.5 math.fmod(xq, 2.) + xql=1.4999999999999716
txq=1605, txq/q=16.05 round(x, 2)=16.05
xq=1608.5 xql=8.526512829121202e-14
abs(xq - txq)=0.5 math.fmod(xq, 2.) + xql=0.5000000000000853
txq=1609, txq/q=16.09 round(x, 2)=16.09
xq=-1605.5 xql=2.842170943040401e-14
abs(xq - txq)=0.5 math.fmod(xq, 2.) + xql=-1.4999999999999716
txq=-1605, txq/q=-16.05 round(x, 2)=-16.05
xq=-1608.5 xql=-8.526512829121202e-14
abs(xq - txq)=0.5 math.fmod(xq, 2.) + xql=-0.5000000000000853
txq=-1609, txq/q=-16.09 round(x, 2)=-16.09


In [ ]:
np.round(56294995342131.5, 2)

In [ ]:
for t in [float, np.half, np.single, np.double, np.longdouble]:
    bs = [np.finfo(t).smallest_normal, np.finfo(t).smallest_subnormal, 
          np.finfo(t).max, t(0.5)]
    xs = [t(np.nextafter(b, d)) for b in bs
          for d in ([np.inf, b, 0] if abs(b) < np.finfo(b).max else
                    [b, 0])]
    cvt = exr_decimal_from_longdouble if t == np.longdouble else float
    lens = [len(decimal.Decimal(cvt(x)).as_tuple().digits)
            for x in xs]
    print(f"{xs[0].__class__.__name__:10} {max(lens):6d} {lens=}")
print("---")
print(f"{np.round(56294995342131.5, 3) == 56294995342131.51 =}")
print(f"{round(56294995342131.5, 3) == 56294995342131.5 =}")
x = 16.055
print(f"{x=}")
print(f"{round(x*10**2)/10**2 == round(x, 2) =}")
print(f"{round(x*10**2)/10**2 == exr_easy_round(x, 2, 'ROUND_HALF_EVEN', fmt_out=False) =}")
print(f"{x=}")
print(f"{math.trunc(x*100)/100 > x =}")
print("--- fma() ---")
for x, n in [(56294995342131.5, 3), (156294995342131.5, 3), (16.055, 2)]:
    print(f"{x=}; {n=}")
    q = 10**n
    xq = x*q
    xql = math.fma(x, q, -x*q)
    print(f"{xq=} {xql=}")
    txq, txql = round(xq), 0
    if txq == xq or abs(xq - txq) == 0.5:
        txq, txql = math.trunc(xq/2)*2, round(math.fmod(xq, 2.) + xql)
    txq, txql = float(txq), float(txql)
    print(f"{(txq + txql)/q=} {(txq + txql)/q == round(x, n)  =}  ({xq=} {xql=})")
    print(f"{txq/q + txql/q=} {txq/q + txql/q == round(x, n)  =}")
    # print(f"{txq=}")
    # print(f"txq={txq :.17f}")
if "float128" in np.__dict__:
    print("--- np.float128() for ndigits <= 3 ---")
    for x, n in [(56294995342131.5, 3), (156294995342131.5, 3), (16.055, 2)]:
        print(f"{x=}; {n=}")
        q = 10**n
        print(f"{float(np.round(np.float128(x)*q)/q) =}")
        # ldxq = np.float128(x)*q
        # xq = float(ldxq)
        # xql = float(ldxq - np.float128(xq))
        # assert np.float128(xq) + np.float128(xql) == ldxq
        # print(f"np.float128(x)*q={ldxq}")
        # print(f"{xq=} {xql=}")
        # print(f"txq={(np.round(np.float128(x)*q)) :.17f}")


In [ ]:
round(x, 2), x, round(x, 2), round(xq)

In [ ]:
round(56294995342131.5, 3)

In [ ]:
round(156294995342131.5, 3)

In [ ]:
# К сожалению, пока Numpy не выдаёт интерфейс к fma(), поэтому точное 
# округление массивов npt.NDArray[np.double] и npt.NDArray[np.longdouble]
# требует техник прошлого века.

# В принципе, для скалярных величин (np.longdouble, ...) можно пробовать 
# округлить результат метода as_integer_ratio(), но это ж как повезёт, с 
# переменным успехом. Плюс, зависит от метода округления.

# T. J. Dekker, A Floating-Point Technique for Extending the Available 
# Precision, 1971
# (6.3) Theorem.
# Jonathan Richard Shewchuk, Adaptive Precision Floating-Point Arithmetic 
# and Fast Robust Geometric Predicates, 1997
# Theorem 17
#
# Значение p-бит плавающего типа (обычно ULP(0.5) = 2**-p или 
# np.finfo(t).negep == p)
# Разделяем на два числа x = hx + hl размерностями: p - s (ahi), 
# 1 (знак) и s - 1 (alo)

_exr_qd_splitter = {
    float:           2**27 + 1,  # negep=53 s=27 -> 26, 1, 26 (4.6e-277..1.3e+300)
    np.float64:      2**27 + 1,  # negep=53 s=27 -> 26, 1, 26 (4.6e-277..1.3e+300)
}
if "float128" in np.__dict__:    # Случай, когда np.longdouble != np.double
    _exr_qd_splitter = { **_exr_qd_splitter,
        np.float128: 2**33 + 1,  # negep=64 s=33 -> 31, 1, 32 (2.9e-4894..1.3e+4922)
    }
if _exr_debug_small_dd:
    _exr_qd_splitter = { **_exr_qd_splitter,
        np.float32:  2**13 + 1,  # negep=24 s=13 -> 11, 1, 12 (8.3e-25..4.1e+34)
        np.float16:  2**6 + 1,   # negep=11 s=6 -> 5, 1, 5 (64.0..1008.0)
                                 # Для np.float16 double-double операции 
                                 # затруднены, поскольку только для 
                                 # t1 = np.float16(64)
                                 # np.finfo(t1).smallest_normal == 
                                 # np.spacing(np.spacing(t1))
                                 # При операциях с меньшими числами могут
                                 # возникать ненормализованные числа
    }

def _exr_np_two_prod(a, b, type_x):
    x = a*b
    # Замена fma(a, b, -x)
    return x, _exr_np_low_dd_mul(a, b, -x)

def _exr_np_low_dd_mul(a, b, mab, type_x):  
    """
    Часть умножения Деккера, которая может быть заменена на `fma(a, b, -a*b)`
    (Предупреждение, это не `fma()`, честнная реализация `fma()` несколько 
    сложнее)
    """
    splitter = _exr_qd_splitter[type_x]
    abig = a*splitter
    bbig = b*splitter
    ahi = (a - abig) + abig
    bhi = (b - bbig) + bbig
    alo = a - ahi
    blo = b - bhi
    return alo*blo + (((mab + ahi*bhi) + alo*bhi) + ahi*blo)

if 'fma' in math.__dict__  and  not _exr_debug_not_fma:
    _exr_math_low_dd_mul = math.fma
else:
    _exr_math_low_dd_mul = _exr_np_low_dd_mul

def _exr_ratio_fma(x, y, z):
    xn, xd = x.as_integer_ratio()
    yn, yd = y.as_integer_ratio()
    zn, zd = z.as_integer_ratio()
    xyd = xd*yd
    return (type(x)((xn*yn*zd + xyd*zn)/(xyd*zd)) 
            if type(x) != _exr_dr_float128_t else
            type(x)(xn*yn*zd + xyd*zn)/type(x)(xyd*zd))

assert _exr_ratio_fma(math.nextafter(1., 0), 
                      math.nextafter(1., 0), 
                      -(1. - math.ulp(1.))) == math.ulp(math.ulp(0.25))
sa, sb = "0.1", "0.2"
for t, sabl in [(float, '-1.6653345369377347e-18'),
                   (np.half, '1.2e-06'),
                   (np.single, '-8.1956386e-10'),
                   (np.double, '-1.6653345369377347e-18'),
                   (np.longdouble, '-7.4538899358378429838e-22')]:
    t1 = t(1.)
    r = _exr_ratio_fma(t(np.nextafter(t1, np.inf)), 
                       t(np.nextafter(t1, np.inf)), 
                       t(-(t1 + 2*np.spacing(t1))))
    assert r == np.spacing(np.spacing(t1))
    assert type(r) == t
    ta, tb = t(sa), t(sb)
    assert t(sabl) == _exr_ratio_fma(ta, tb, -ta*tb)
    if t != np.longdouble:
        assert t(math.fma(ta, tb, -ta*tb)) == _exr_ratio_fma(ta, tb, -ta*tb)
        if _exr_math_low_dd_mul != _exr_np_low_dd_mul or t in _exr_qd_splitter:
            assert t(sabl) == t(_exr_math_low_dd_mul(ta, tb, -ta*tb))
    if t in _exr_qd_splitter:
        assert t(sabl) == _exr_np_low_dd_mul(ta, tb, -ta*tb, type_x=t)

In [ ]:
def _exr_fmt_out_ratio_trunc(x, ndigits, type_x):
    q = 10**ndigits
    n, d = x.as_integer_ratio()
    return type_x(math.trunc((q*n)/d))/q

def _exr_fmt_out_math_trunc(x, ndigits, type_x):
    q = float(10**ndigits)
    xq = x*q
    xql = _exr_math_low_dd_mul(x, q, -xq)
    txq = math.trunc(xq)
    if txq == xq:
        txq += math.ceil(xql) if 0 <= xq else math.trunc(xql)
    return txq/q

def nocvt(x):
    return x

def _exr_fmt_out_np_trunc(x, ndigits, type_x):
    if type_x == np.longdouble:
        domain = np
        type_i = type_x
        cvt = nocvt
    else:
        domain = math
        type_i = float
        cvt = type_x
        x = type_i(x)

    q = type_i(10**ndigits)
    xq = x*q
    xql = _exr_np_low_dd_mul(x, q, -xq, type_x=type_x)
    txq = domain.trunc(xq)
    if txq == xq:
        txq += domain.ceil(xql) if 0 <= xq else domain.trunc(xql)
    return cvt(txq/q)

(_exr_fmt_out_ratio_trunc(4.00001, 5, type_x=float),
 _exr_fmt_out_math_trunc(4.00001, 5, type_x=float),
 _exr_fmt_out_np_trunc(4.00001, 5, type_x=float)
)
sx = 4.00001
n = 5
for t in [float, np.double, np.longdouble]:
    print(t)
    x = t(sx)
    print(t.__class__.__name__, 
          [f(x, n, type_x=t) 
           for f in ([_exr_fmt_out_ratio_trunc, 
                      _exr_fmt_out_np_trunc, 
                     ] +
                     ([_exr_fmt_out_math_trunc] if t != np.longdouble else []))
          ])


In [ ]:

# t = np.double
# for t in [float, np.double, np.longdouble]:
#     print(t)
#     x = t('4.00001')
#     %timeit t(t(x))
#     %timeit t(t(t(x)))
#     %timeit nocvt(nocvt(x))
#     %timeit nocvt(nocvt(nocvt(x)))

In [ ]:
sx = 4.00001
n = 5
for t in [float, np.double, np.longdouble]:
    print(t)
    x = t(sx)
    for f in ([_exr_fmt_out_ratio_trunc, 
               _exr_fmt_out_np_trunc, 
              ] +
              ([_exr_fmt_out_math_trunc] if t != np.longdouble else [])):
        print(f)
        %timeit f(x, n, type_x=t)
print('Хорь')

In [ ]:
x = float(3.3)
%timeit _exr_qd_splitter[type(x)]
x = np.double(3.3)
%timeit _exr_qd_splitter[type(x)]
x = np.longdouble(3.3)
%timeit _exr_qd_splitter[type(x)]

In [ ]:
_exr_qd_splitter[float]

In [ ]:
_exr_ratio_fma(math.nextafter(1., 0), 
                      math.nextafter(1., 0), 
                      -(1. - math.ulp(1.)))

In [ ]:
for t in [float, np.single, np.double, np.longdouble]:
    a, b = t('0.1'), t('0.2')
    mab = -a*b
    print(t)
    if t in _exr_qd_splitter:
        %timeit _exr_np_low_dd_mul(a, b, mab, type_x=t)
    %timeit _exr_ratio_fma(a, b, mab)
    if t != np.longdouble:
        %timeit _exr_math_low_dd_mul(a, b, mab)
print('Хорь')

In [ ]:
cnt = 0
# minx = np.float16(0.25)  # np.float16(0)
# maxx = np.float16(4)  # np.float16(1008.0)
minx = np.float16(2)  # np.float16(0)
maxx = np.float16(4)  # np.float16(1008.0)
x0 = minx
while x0 < maxx:
    for x in [x0, -x0]:
        split(x, type(x))
    x0 = np.nextafter(x0, np.inf)
    cnt += 1
cntx0 = cnt
print(cntx0)

In [ ]:
import tqdm 

In [ ]:
cnt = 0
x0 = minx
type_x = type(x)
with tqdm.tqdm(total=cntx0) as pbar:
    while x0 < maxx:
        y0 = minx
        while y0 < maxx:
            z0 = minx
            while z0 < maxx:
                # for x, y in [(x0, y0), (x0, -y0), (-x0, y0)]:
                x, y, z = x0, y0, z0
                xyl = fma_nofma(x, y, z, type_x=type_x)
                #assert xyl == type_x(math.fma(x, y, z))
                if xyl != type_x(math.fma(x, y, z)):
                    print(f"{x=}, {y=}, {z=}, {xyl=}")
                    break
                # xyh = x*y
                # xyl = _exr_np_low_dd_mul(x, y, -xyh, type_x=type_x)
                # assert xyl == type_x(math.fma(x, y, -xyh))
                # xyh, xyl = two_prod(x, y, type_x=type_x)
                # assert type_x == type(xyh) and type_x == type(xyl)
                # assert xyh == xyh + xyl  and  abs(xyl) < np.spacing(xyh)
                # assert float(xyh) + float(xyl) == float(x)*float(y)
                z0 = np.nextafter(z0, np.inf)
                cnt += 1
            y0 = np.nextafter(y0, np.inf)
        # assert type_x == type(xyh) and type_x == type(xyl)
        x0 = np.nextafter(x0, np.inf)
        pbar.update()
print(cnt)

In [ ]:
x, y, z, fma_nofma(x, y, z, type_x=type_x), type_x(math.fma(x, y, z)), math.fma(x, y, z), type_x(float(x)*float(y) + float(z))

In [ ]:
_two_prod(np.float16(0.5083), np.float16(0.6177), type_x=type_x)

In [ ]:
np.spacing(np.float16(0.314)), np.spacing(np.spacing(np.float16(0.314)))

In [ ]:
mh, ml = _two_prod(x, y, type_x=type_x)
mh, ml, (float(x)*float(y) - (float(mh) + float(ml)))

In [ ]:
fh, fl = _two_sum(z, mh)
fh, fl, float(fh) + float(fl), type_x(float(fh) + float(fl)), type_x(float(fh) + float(fl) + float(ml))

In [ ]:
float(fl + ml), float(fl) + float(ml), type_x(float(fl) + float(ml))

In [ ]:
fl += ml
fh, fl

In [ ]:
fh, fl = _two_sum_quick(fh, fl)
fh, fl

In [ ]:
0.2505 + 2.4e-07 + float(z)

In [ ]:
x, y, xyh, xyl, abs(xyl)/np.spacing(xyh)

In [ ]:
xyh, xyh + xyl 

In [ ]:
(float(xyh) - float(xyl)) - float(x)*float(y)

In [ ]:
xyl, type_x(math.fma(x, y, -xyh))

In [ ]:
_exr_np_low_dd_mul(x, y, -xyh, type_x=type_x)

In [ ]:
_exr_int_sw = dict()  # Таблица функций целого округления
_exr_v0r_round_accepted_types = (float,  # decimal.Decimal, fractions.Fraction,
                                 np.half, np.single, np.double)
_exr_v0r_np_domain_types = (np.ndarray, _exr_dr_float128_t)
_exr_v0r_ext_types = {np.half: np.single, np.single: np.double}
# Методы округления для которых гарантируется определённый порядок
_exr_v0r_strict_roundings = set((decimal.ROUND_CEILING, decimal.ROUND_DOWN, 
                                 decimal.ROUND_FLOOR, decimal.ROUND_UP))

def exr_v0_round(x, ndigits=None, fmt_out=None, 
                 rounding=decimal.ROUND_HALF_UP):
    type_x = type(x)
    if type_x == np.ndarray:
        type_x = x.dtype.type
        if not x.shape:
            x = type_x(x)
    if (rounding == decimal.ROUND_HALF_UP and fmt_out != False and 
        isinstance(x, _exr_v0r_round_accepted_types)):
        return round(x, ndigits=ndigits)
    domain = np if isinstance(x, _exr_v0r_np_domain_types) else math
    round_int = _exr_int_sw[rounding]
    if ndigits is None:
        return round_int(x, domain=domain)
    if not math.isfinite(x):
        return x
    if ndigits <= 0:
        q = type_x(10**-ndigits)
        return round_int(x/q, domain=domain)*q
    q = type_x(10**ndigits)
    if fmt_out is None  and  roundings not in _exr_v0r_strict_roundings:
        # В принципе, для обеспечения гарантии упорядочености, можно было бы
        # использовать:
        # xf, xi = modf(x)
        # return (xi*q + round_int(xf*q, domain=domain))/q
        # 
        # Что увеличило бы точность, как при больших x, так и для 
        # ndigits <= 8, но fma() использует столько же сложений и 
        # умножений, и делает это немного быстрее, кроме того обеспечивает 
        # абсолютно точный результат.

        return round_int(x*q, domain=domain)/q
    if domain == math:
        # two_product(x, q)
        xqh = x*q
        xql = _exr_math_low_dd_mul(x, q, -xq)  # math.fma(x, q, -xq)
        assert xqh = xqh + xql  and  abs(xql) < 0.5*math.ulp(xqh)
        if fmt_out == False:
            # xql сравним с (q/2)*math.ulp(x) <= math.ulp(xq)
            xql += math.copysign((q/2)*math.ulp(x), x)
            # two_sum_quick
            t = xqh
            xqh += xql
            xql = xql - (xqh - t)
            assert xqh = xqh + xql  and  abs(xql) < 0.5*math.ulp(xqh)
        rxq = round_int(xqh)
        if rxq == xq:
            rxq += round_int(xql)
        return rxq/q
    ext_t = _exr_v0r_ext_types.get(type_x)
    if ext_t is None:
        # two_product(x, q)
        xqh = x*q
        xql = _exr_np_low_dd_mul(x, q, -xq, type_x=type_x)  # Грядущий np.fma(x, q, -xq)
        np.testing.assert_array_equal(xqh, xqh + xql)
        np.testing.assert_array_less(xql, 0.5*np.spacing(xqh))
        if fmt_out == False:
            # xql сравним с (q/2)*math.ulp(x) <= math.ulp(xq)
            xql += np.copysign((q/2)*np.spacing(x), x)
            # two_sum_quick
            t = xqh
            xqh += xql
            xql = xql - (xqh - t)
            np.testing.assert_array_equal(xqh, xqh + xql)
            np.testing.assert_array_less(xql, 0.5*np.spacing(xqh))
        rxq = round_int(xqh, domain=np)
        return np.where(rxq == xq, rxq + round_int(xql, domain=np), rxq)/q
    # Для np.half, np.single и массивов на их основе
    ex = ext_t(x)
    if fmt_out == False:
        ex += np.copysign((q/2)*np.spacing(x), x)
    return round_int(x*q, domain=domain)/q


In [ ]:
x = {3: 4}
x.get(3) is None

In [ ]:
decimal.ROUND_HALF_EVEN not in _exr_v0r_strict_roundings

In [ ]:
x = np.array(np.single(1.23))
x.dtype, x.dtype.type

In [ ]:
f = 1.23
%timeit round(f)
%timeit round(float(f))
%timeit round(float(x))

In [ ]:
bool(x.shape)

In [ ]:
        # 
        # Причины использования modf() или почему не стоит 
        # использовать math.trunc(x*q)/q
        # q = 100
        # for x in [8589934591.059999, 2147483647.8799999, 536870911.96999997,
        #           268435455.76999998, 67108863.949999996]:
        #     txqq = math.trunc(x*q)/q
        #     xqq = x*q/q
        #     print(f"{txqq < x=} {x=} {txqq=} {xqq=}")
        # 
        # TODO: Обосновать, что для xf < 1, math.trunc(xf*q)/q <= xf
        # или это я просто не нашёл контрпримера?
        #
        # Проблема xi + math.trunc(xf*q)/q, для n=10, q=10**10
        # for n, x, hm in [(10, 524287.9999291525, 524287.9999291524),
        #                  (10, 262143.99998263217, 262143.9999826321)]:
        #     q = 10**n
        #     xf, xi = math.modf(x)
        #     r = xi + math.trunc(xf*q)/q
        #     print(math.trunc(xf*q), math.trunc(xf*q)/q, r)
        #     print(math.modf(r))
        #     print(math.modf(hm))
        #     print((x - hm)/math.ulp(hm))
        #
        # TODO: гипотеза, что для ndigits <= sys.float_info.dig//2 
        # такого быть не может 
        #
        # np.round(56294995342131.5, 3)


In [ ]:
x=67108863.949999996; math.trunc(x*100)/100 > x

In [ ]:
x=67108863.949999996
print(f"{x=}; {math.trunc(x*100)/100 > x =}")

In [ ]:
q = 100
for x in [8589934591.059999, 2147483647.8799999, 536870911.96999997,
          268435455.76999998, 67108863.949999996]:
    txqq = math.trunc(x*q)/q
    xqq = x*q/q
    print(f"{txqq < x=} {x=} {txqq=} {xqq=}")


In [ ]:
import math 

def exr_ceil_base(x):
    """
    Базовая реализация округления к большему, аналогичного `math.ceil()`, 
    IEEE 754: roundTowardNegative, ROUND_CEILING.
    """
    return x - x%-1.
    # Варианты:
    # x - x%-1.
    # -(x//-1.)
    # float(math.ceil(x))

def exr_trunc_base(x):
    """
    Базовая реализация округления к меньшему по модулю, аналогична 
    `math.trunc()` и `int()`, IEEE 754: roundTowardZero (усечение к 0);, 
    ROUND_DOWN.
    """
    return x - math.fmod(x, 1.)
    # Варианты:
    # x - math.fmod(x, 1.)
    # float(math.trunc(x))
    # math.modf(x)[1]
    # x - math.modf(x)[0]

def exr_floor_base(x):
    """
    Базовая реализация округление к меньшему, аналогична `math.floor()`, 
    IEEE 754: roundTowardNegative, ROUND_FLOOR.
    """
    return x//1.
    # Варианты:
    # x//1.
    # x - x%1.
    # float(math.floor(x))
    # divmod(x, 1.)[0]

def exr_sql_round_base(x):
    """
    Базовая реализация округления до ближашего большего по модулю, 
    аналогична функции C/C++/SQL/... `round()`, IEEE 754: roundTiesToAway 
    (математическое округление), ROUND_HALF_UP.
    """
    return float(math.trunc(x + math.copysign(0.5, x)))
    # float(math.trunc(x + math.copysign(0.5, x)))
    # xf, xi = math.modf(x); xi + ((0.5 <= xf) if 0 <= xf else -(0.5 <= -xf))
    # xf, xi = math.modf(x); xi + math.copysign((0.5 <= abs(xf)), xf)

def exr_half_down_base(x):
    """
    Базовая реализация округления до ближашего меньшего по модулю, 
    ROUND_HALF_DOWN.
    """        
    return float(math.ceil(x - 0.5)) if 0 <= x else float(math.floor(x + 0.5))
    # Варианты:
    # float(math.ceil(x - 0.5)) if 0 <= x else float(math.floor(x + 0.5))



"""
        ROUND_HALF_DOWN - округление до ближашего меньшего по модулю, 
                          широкораспространённых аналогов не имеет, 
                          в IEEE 754 не входит;
        ROUND_UP        - округление к большему по модулю, 
                          широкораспространённых аналогов не имеет, 
                          в IEEE 754 не входит;
        ROUND_05UP      - округление к меньшему по модулю, если последняя 
                          цифра результата не получается 0 или 5, в этом 
                          случае округление к большему по модулю,
                          широкораспространённых аналогов не имеет, 
                          в IEEE 754 не входит;
"""
for f, r in [(exr_ceil_base, decimal.ROUND_CEILING),
             (exr_trunc_base, decimal.ROUND_DOWN),
             (exr_floor_base, decimal.ROUND_FLOOR),
             (exr_sql_round_base, decimal.ROUND_HALF_UP), 
             (exr_half_down_base, decimal.ROUND_HALF_DOWN),
             #(exr_up_int, decimal.ROUND_UP),
             #(exr_05up_int, decimal.ROUND_05UP),
            ]:
    for x in np.arange(-2., 2., 0.25):
        assert f(x) == exr_decimal_round(x, rounding=r), f"{f=} {r=} {x=} {f(x)=}"
        for i in [-math.inf, math.inf]:
            y = x
            for _ in range(3):
                y = math.nextafter(y, i)
                assert f(x) == exr_decimal_round(x, rounding=r), f"{f=} {r=} {x=} {f(x)=}"
print("Хорь")

In [ ]:
def exr_floor_base(x):
    return x//1.

def exr_sql_round_base(x):
    return float(math.trunc(x + math.copysign(0.5, x)))

def exr_floor_(x, ndigits=0):
    if ndigits <= 0:
        q = 10**(-ndigits)
        return ((x/q)//1.)*q
    q = 10**ndigits
    return ((x*q)//1.)/q

def exr_sql_round_(x, ndigits=0):
    if ndigits <= 0:
        q = 10**(-ndigits)
        return math.trunc(x/q + math.copysign(0.5, x))*q
    q = 10**ndigits
    return float(math.trunc(x*q + math.copysign(0.5, x)))/q



exr_sw = {decimal.ROUND_HALF_UP: exr_sql_round_base,
          decimal.ROUND_FLOOR: exr_floor_base
         }
def exr_basetest(x, ndigits=0, rounding=decimal.ROUND_HALF_UP):
    if ndigits <= 0:
        q = 10**(-ndigits)
        return exr_sw[rounding](x/q)*q
    q = 10**ndigits
    return exr_sw[rounding](x*q)/q


In [ ]:
%timeit exr_basetest(2.0115, 3, rounding=decimal.ROUND_HALF_UP)
%timeit exr_sql_round_(2.0115, 3)

In [ ]:
%timeit exr_basetest(2.0115, 3, rounding=decimal.ROUND_FLOOR)
%timeit exr_floor_(2.0115, 3)

In [ ]:
x, f(x), exr_decimal_round(x, rounding=r)

In [ ]:
divmod(x, 1.), x//1.

In [ ]:
x = -3.75
math.modf(x), divmod(x, 1), x - math.ceil(x)

In [ ]:
np.round(56294995342131.5, 3)

In [ ]:
xf, xi = np.modf(56294995342131.5)
xi + np.round(xf, 3)

In [ ]:
x = 56294995342131.5
q = 10**3
math.ceil(x*q)/q

In [ ]:
np.log2(x) + np.log2(q)

In [ ]:
np.round?

In [ ]:
np.fix([2.1, 2.9, -2.1, -2.9])

In [ ]:
np.trunc([2.1, 2.9, -2.1, -2.9])

In [ ]:
x = 2.654

%timeit x - math.fmod(x, 1.)
%timeit math.modf(x)[1]
%timeit float(math.trunc(x))
%timeit x - math.modf(x)[0]


In [ ]:
%timeit x - math.modf(x)[0]
%timeit float(math.trunc(x))
%timeit x - math.fmod(x, 1.)

In [ ]:
math.modf(x)

In [ ]:
import dis
dis.dis(lambda x: x//1.)

In [ ]:
dis.dis(lambda x: (-(x//-1.)))

In [ ]:
x = 2.654

%timeit x - x%-1.
%timeit (-(x//-1.))
%timeit float(math.ceil(x))
%timeit x - math.remainder(x, -1.)


In [ ]:
xf, xi = math.modf(x)
xf, xi

In [ ]:
x = 2.654
%timeit math.modf(x)[1]
%timeit x//1.
%timeit float(math.trunc(x))

In [ ]:
x = 2.654
%timeit x//1. if 0 <= x else x//-1.
%timeit float(math.trunc(x)) if math.isfinite(x) else x

In [ ]:
math.isfinite(x)

In [ ]:
t = 1.9
exr_decimal_round(t, rounding=r), exr_05up_int(t)

In [ ]:
np.arange(-2., 2., 0.2)

In [ ]:
x = np.arange(-2., 2., 0.25)
len(x), x[len(x)//2]

In [ ]:
import math
math.modf(2.1)

In [ ]:
n = 17
x = np.array(np.longdouble("0." + "2"*n + "5"))
print(x)
print(exr_decimal_round(x, n))
print(exr_decimal_round(x, n, str_inp=True))
print(exr_decimal_round(x, n, str_inp=False))

In [ ]:
repr(x)

In [ ]:
x = (np.double("0." + "2"*n + "5"))
print(x)
print(exr_decimal_round(x, n))
print(exr_decimal_round(x, n, str_inp=True))
print(exr_decimal_round(x, n, str_inp=False))

In [ ]:
exr_decimal_round(np.longdouble(5)/np.longdouble(9), 18, str_inp=False)

In [ ]:
np.longdouble(5)/np.longdouble(9)

In [ ]:
decimal.Decimal(str(np.longdouble(5)/np.longdouble(9)))

In [ ]:
nld59 = np.longdouble(5)/np.longdouble(9)
snld59 = str(nld59)
np.longdouble(snld59) - nld59

In [ ]:
assert str(decimal.Decimal(snld59)) == snld59
dsnld59 = decimal.Decimal(snld59)
dsnld59

In [ ]:
inld59 = nld59.as_integer_ratio()
dinld59 = decimal.Decimal(inld59[0])/decimal.Decimal(inld59[1])
dinld59, dinld59 - dsnld59, (np.nextafter(nld59, np.inf) - nld59)/2

In [ ]:
np.longdouble(str(dinld59)), np.longdouble(str(dsnld59))

In [ ]:
exr_decimal_to_longdouble(exr_decimal_from_longdouble(nld59))

In [ ]:
nld59.__format__(".18g"), nld59.__repr__()

In [ ]:

with np.printoptions(precision=8):
    print(np.array([nld59]))

In [ ]:
nld59.as_integer_ratio()

In [ ]:
decimal.Decimal(10248191152060862009)/decimal.Decimal(18446744073709551616)

In [ ]:
f = 0.3
nd = np.double(0.3)
ad = np.array(nd)
nld = np.longdouble(0.3)
ald = np.array(nld)
type(f), type(nd), type(ad), type(nld), type(ald)

In [ ]:
np.isreal(f), np.isreal(nd), np.isreal(ad), np.isreal(nld), np.isreal(ald)

In [ ]:
np_longdouble_type = np.longdouble
np_longdouble_types = (np.longdouble, np.ndarray)
x = nld
%timeit isinstance(x, np_longdouble_type)
%timeit isinstance(x, np_longdouble_type) or (isinstance(x, np.ndarray) and x.dtype == np_longdouble_type)
%timeit isinstance(x, np_longdouble_types) and x.dtype == np_longdouble_type


In [ ]:
x = nd
%timeit isinstance(x, np_longdouble_type)
%timeit isinstance(x, np_longdouble_type) or (isinstance(x, np.ndarray) and x.dtype == np_longdouble_type)
%timeit isinstance(x, np_longdouble_types) and x.dtype == np_longdouble_type

In [ ]:
x = f
%timeit isinstance(x, np_longdouble_type)
%timeit isinstance(x, np_longdouble_type) or (isinstance(x, np.ndarray) and x.dtype == np_longdouble_type)
%timeit isinstance(x, np_longdouble_types) and x.dtype == np_longdouble_type

In [ ]:
%timeit np.finfo(f).bits
%timeit np.finfo(f).nmant

In [ ]:
tt = (np.float16, np.float32, np.ndarray)

In [ ]:
decimal_accepted_types = (float, np.double, decimal.Decimal, str)
np_longdouble_type = np.longdouble
np_longdouble_from = lambda ld: decimal.Decimal(str(ld))
np_longdouble_to = lambda d: np.longdouble(str(d))

def tcvt(x, ndigits=None, str_inp=None):
    x_to = type(x)
    if x_to == np.ndarray:
        x_to = x.dtype.type
        x = x_to(x)
    if str_inp != False or ndigits <= 0:
        pre_cvt_x = str(x)
        if x_to == _exr_dr_float128_t:
            x_to = exr_decimal_to_longdouble
    elif isinstance(x, _exr_dr_decimal_accepted_types):
        pre_cvt_x = x
    elif x_to == _exr_dr_float128_t:
        pre_cvt_x = exr_decimal_from_longdouble(x)
        x_to = exr_decimal_to_longdouble
    else:
        pre_cvt_x = float(x)
    return x_to(pre_cvt_x)

def tcvt1(x, ndigits=None, str_inp=None):
    x_to = type(x)
    if x_to == np.ndarray:
        x_to = x.dtype.type
        x = x_to(x)
    if str_inp != False or ndigits <= 0:
        pre_cvt_x = str(x)
        if x_to == _exr_dr_float128_t:
            x_to = exr_decimal_to_longdouble
    elif x_to in _exr_dr_decimal_accepted_types:
        pre_cvt_x = x
    elif x_to == _exr_dr_float128_t:
        pre_cvt_x = exr_decimal_from_longdouble(x)
        x_to = exr_decimal_to_longdouble
    else:
        pre_cvt_x = float(x)
    return x_to(pre_cvt_x)

# def tcvt2(x, ndigits=None, str_inp=None):
#     if isinstance(x, np.ndarray):
#         x = x.dtype.type(x)
#     x_to = type(x)
#     return x_to(x)

ndigits = 2
str_inp = False

for t in (float, np.double, decimal.Decimal, str, np.half, np.longdouble,
          lambda x: np.array(np.double(x))):
    x = t(2.0115)
    print(f"    {t=}, {tcvt(x, ndigits=ndigits, str_inp=str_inp)=}")
    %timeit tcvt(x, ndigits=ndigits, str_inp=str_inp)
    print(f"    {t=}, {tcvt1(x, ndigits=ndigits, str_inp=str_inp)=}")
    %timeit tcvt1(x, ndigits=ndigits, str_inp=str_inp)
    # print(f"    {t=}, {tcvt1(x, ndigits=ndigits, str_inp=str_inp)=}")
    # %timeit tcvt2(x, ndigits=ndigits, str_inp=str_inp)
print("Хорь")

In [ ]:
np.longdouble(decimal.Decimal(np.inf)) 

In [ ]:
x = np.longdouble(np.pi)
l = np.ldexp(x, 32)
m = x*(2**32)
print(f"{l == m=} {l=} {m=}")
%timeit l = np.ldexp(x, 32)
%timeit m = x*(2**32)

In [ ]:
x = np.longdouble(np.pi)
l = np.ldexp(x, -32)
m = x/(2**32)
print(f"{l == m=} {l=} {m=}")
%timeit l = np.ldexp(x, -32)
%timeit m = x/(2**32)

In [ ]:
np.fix?

In [ ]:
tt = (float, np.double, decimal.Decimal, str)
str_inp = None
for t in (float, np.double, decimal.Decimal, str, np.half, np.longdouble):
    x = 2.0115
    cx = x if isinstance(x, tt) else float(x)
    print(t, cx)
    %timeit cx = x if str_inp != False or isinstance(x, tt) else float(x)

In [ ]:
import math

def _exr_two_sum_quick(x, y):
    r = x + y
    e = y - (r - x)
    return r, e

def _exr_nofma_two_product(x, y):
    def _two_product(x, y):
        u = x*134217729.0
        v = y*134217729.0
        s = u - (u - x)
        t = v - (v - y)
        f = x - s
        g = y - t
        r = x*y
        e = ((s*t - r) + s*g + f*t) + f*g
        return r, e

if 'fma' in math.__dict__  and  not _exr_debug_not_fma:
    def _exr_fma_two_product(x, y):
        r = x*y
        e = math.fma(x, y, -r)
        return r, e

    

In [ ]:
134217729.0 .hex()

In [ ]:
float.fromhex('0x1.0000002000000p+27')

In [ ]:
_exr_debug = True
_exr_ndebug = False

%timeit if _exr_debug: assert math.sin(1) < 1
%timeit assert _exr_ndebug or (math.sin(1) < 1)

In [ ]:
sys.float_info

In [ ]:
import math

_exr_ndebug = False
_exr_debug_not_fma = False

def _exr_quick_two_sum(a, b):
    assert _exr_ndebug or (abs(a) >= abs(b))
    s = a + b
    err = b - (s - a)
    assert _exr_ndebug or (s == s + err and abs(err) <= 0.5*math.ulp(s))
    return s, err

_exr_nftp_splitter = 2**27 - 1
_exr_nftp_splitter_tresh = 2**(1024 - 1 - 27)

def _exr_notfma_two_prod(a, b):
    assert _exr_ndebug or (abs(a) < _exr_nftp_splitter_tresh and 
                           abs(b) < _exr_nftp_splitter_tresh)
    p = a*b
    t = _exr_nftp_splitter*a
    a_hi = t - (t - a)
    a_lo = a - a_hi
    t = _exr_nftp_splitter*b
    b_hi = t - (t - b)
    b_lo = b - b_hi
    err = ((a_hi*b_hi - p) + a_hi*b_lo + a_lo*b_hi) + a_lo*b_lo
    assert _exr_ndebug or (p == p + err and abs(err) <= 0.5*math.ulp(p))
    return p, err

_exr_two_prod = _exr_notfma_two_prod

if 'fma' in math.__dict__  and  not _exr_debug_not_fma:
    def _exr_fma_two_prod(a, b):
        p = a*b
        err = math.fma(a, b, -p)
        assert _exr_ndebug or (p == p + err and abs(err) <= 0.5*math.ulp(p))
        return p, err

    _exr_two_prod = _exr_fma_two_prod

In [ ]:
def _exr_dd_ceil(h, l):
    r = math.ceil(h)
    if r != h:
        return r
    return r + math.ceil(l)

def _exr_dd_floor(h, l):
    r = math.floor(h)
    if r != h:
        return r
    return r + math.floor(l)

def _exr_dd_trunc(h, l):
    r = math.trunc(h)
    if r != h:
        return r
    return r + math.floor(l) if h > 0 else math.ceil(l)

def _exr_dd_round(h, l):
    r = round(h)
    if r == h:
        return r + round(l)
    if abs(r - h) == 0.5 and l < 0.:
        r -= 1
    return r

def _exr_dd_sql_round(h, l):
    r = round(h)
    if r == h:
        return r + round(l)
    if abs(r - h) == 0.5 and l < 0.:
        r -= 1
    return r


    def exr_math_trunc(x, ndigits=None, str_inp=None):
        """
        Усечение к меньшему по модулю (rounding=decimal.ROUND_DOWN)
        
        >>> exr_math_trunc(0.21, 2)
        0.21
        >>> exr_math_trunc(0.21, 2, str_inp=False)
        0.2
        >>> exr_math_trunc(0.21, 2, str_inp=True)
        0.21
        >>> exr_math_trunc(2.01, 2)
        2.0
        >>> exr_math_trunc(2.01, 2, str_inp=False)
        2.0
        >>> exr_math_trunc(2.01, 2, str_inp=True)
        2.01
        >>> exr_math_trunc(2.01)
        2
        >>> exr_math_trunc(201.0, -2)
        200.0
        >>> exr_math_trunc(math.nan, -2)
        nan
        >>> exr_math_trunc(math.inf, -2)
        inf
        """
        if ndigits is None:
            return math.trunc(x)
        if not math.isfinite(x):
            return x
        if ndigits <= 0:
            q = 10**-ndigits
            return type(x)(math.trunc(x/q)*q)
        q = 10**ndigits
        if str_inp is None:
            # Причины использования modf() или почему не стоит 
            # использовать math.trunc(x*q)/q
            # q = 100
            # for x in [8589934591.059999, 2147483647.8799999, 536870911.96999997,
            #           268435455.76999998, 67108863.949999996]:
            #     txqq = math.trunc(x*q)/q
            #     xqq = x*q/q
            #     print(f"{txqq < x=} {x=} {txqq=} {xqq=}")
            # 
            # TODO: Обосновать, что для xf < 1, math.trunc(xf*q)/q <= xf
            # или это я просто не нашёл контрпримера?
            #
            # Проблема xi + math.trunc(xf*q)/q, для n=10, q=10**10
            # for n, x, hm in [(10, 524287.9999291525, 524287.9999291524),
            #                  (10, 262143.99998263217, 262143.9999826321)]:
            #     q = 10**n
            #     xf, xi = math.modf(x)
            #     r = xi + math.trunc(xf*q)/q
            #     print(math.trunc(xf*q), math.trunc(xf*q)/q, r)
            #     print(math.modf(r))
            #     print(math.modf(hm))
            #     print((x - hm)/math.ulp(hm))
            #
            # TODO: гипотеза, что для ndigits <= sys.float_info.dig//2 
            # такого быть не может 
            xf, xi = math.modf(x)
            return type(x)(xi + math.trunc(xf*q)/q)
        xqh, xql = _exr_two_product(x, q)
        if str_inp:
            xql += math.copysign((q//2)*math.ulp(x), x)
            xqh, xql = _exr_two_sum_quick(xqh, xql) 
        return type(x)(_exr_dd_trunc(xqh, xql)/q)
else:
    # Множитель: 1 + eps/2
    _exr_mt_1eps2_n, _exr_mt_1eps2_d = math.ulp(0.5).as_integer_ratio()
    _exr_mt_1eps2_n += _exr_mt_1eps2_d
    
    def exr_math_trunc(x, ndigits=None, str_inp=None):
        """
        Усечение к меньшему по модулю (rounding=decimal.ROUND_DOWN)
        
        >>> exr_math_trunc(0.21, 2)
        0.21
        >>> exr_math_trunc(0.21, 2, str_inp=False)
        0.2
        >>> exr_math_trunc(0.21, 2, str_inp=True)
        0.21
        >>> exr_math_trunc(2.01, 2)
        2.0
        >>> exr_math_trunc(2.01, 2, str_inp=False)
        2.0
        >>> exr_math_trunc(2.01, 2, str_inp=True)
        2.01
        >>> exr_math_trunc(2.01)
        2
        >>> exr_math_trunc(201.0, -2)
        200.0
        >>> exr_math_trunc(math.nan, -2)
        nan
        >>> exr_math_trunc(math.inf, -2)
        inf
        """
        if ndigits is None:
            return math.trunc(x)
        if not math.isfinite(x):
            return x
        if ndigits <= 0:
            q = 10**-ndigits
            return type(x)(math.trunc(x/q)*q)
        q = 10**ndigits
        if str_inp is None:
            tx = math.trunc(x)
            return type(x)(tx + math.trunc((x - tx)*q)/q)
        n, d = abs(x).as_integer_ratio()
        n *= q
        if str_inp:
            n *= _exr_mt_1eps2_n
            d *= _exr_mt_1eps2_d
        return type(x)(math.copysign((n//d)/q, x))


In [ ]:
import doctest
doctest.testmod()

In [ ]:
import sys
sys.float_info.dig//2

In [ ]:
assert False

In [ ]:
n = 9
q = 10**n
up0 = 1/math.ulp(1.)/q
up = 2.**(math.floor(math.log2(up0)) + 10)
u = math.nextafter(up, math.inf)
err_gt_u = 0
lst_err_gt_u = []
err_lt_m = 0
lst_err_lt_m = []
for i in range(10_000_000):
    m = exr_round(u, n, str_inp=False, rounding=decimal.ROUND_DOWN)
    h = exr_round(u, n, str_inp=True, rounding=decimal.ROUND_DOWN)
    assert exr_math_trunc(u, n, str_inp=False) == m and abs(m) <= abs(u) 
    assert exr_math_trunc(u, n, str_inp=True) == h and h <= abs(u)
    assert m <= h
    t = exr_math_trunc(u, n)
    assert t <= math.nextafter(u, math.inf), f"{u=} {h=} {m=} {t=} {(t - u)/math.ulp(t):.1f}"
    assert t >= math.nextafter(m, 0), f"{u=} {h=} {m=} {t=} {(t - u)/math.ulp(t):.1f}"
    if t > u:
        err_gt_u += 1
        lst_err_gt_u.append((u, m, h, t))
    elif t < m:
        err_lt_m += 1
        lst_err_lt_m.append((u, m, h, t))
    else:
        assert t == m or t == h, f"{u=} {h=} {m=} {t=} {(t - m)/math.ulp(t):.1f}"
    u = math.nextafter(u, 0)
print(f"{err_gt_u=} {err_gt_u/(i+1)=}")
print(f"{err_lt_m=} {err_lt_m/(i+1)=}")
print(f"{(err_gt_u + err_lt_m)/(i+1)=}")
i+1

In [ ]:
q, n

In [ ]:
for n, x, hm in [(10, 524287.9999291525, 524287.9999291524),
                 (10, 262143.99998263217, 262143.9999826321)]:
    q = 10**n
    xf, xi = math.modf(x)
    r = xi + math.trunc(xf*q)/q
    print(math.trunc(xf*q), math.trunc(xf*q)/q, r)
    print(math.modf(r))
    print(math.modf(hm))
    print((x - hm)/math.ulp(hm))

In [ ]:
math.log2(9999826321) + 10*math.log2(10)

In [ ]:
ex = [
    (8589934591.059999, 8589934591.05, 8589934591.05, 8589934591.06),
    (2147483647.8799999, 2147483647.87, 2147483647.87, 2147483647.88),
    (536870911.96999997, 536870911.96, 536870911.96, 536870911.97),
    (268435455.76999998, 268435455.76, 268435455.76, 268435455.77)
    (67108863.949999996, 67108863.94, 67108863.94, 67108863.95)
]

In [ ]:
math.log2(67108863.949999996*100)

In [ ]:
q = 100
for x in [8589934591.059999, 2147483647.8799999, 536870911.96999997,
          268435455.76999998, 67108863.949999996]:
    txqq = math.trunc(x*q)/q
    xqq = x*q/q
    print(f"{txqq < x=} {x=} {txqq=} {xqq=}")

In [ ]:
assert False

In [ ]:
import doctest
doctest.testmod()

In [ ]:
%timeit exr_math_trunc(2.01, 2, str_inp=True)
%timeit exr_ratio_trunc(2.01, 2, str_inp=True)

In [ ]:
x, n = 0.21, 2
exr_math_trunc(x, n), exr_math_trunc(x, n, str_inp=False), exr_math_trunc(x, n, str_inp=True)

In [ ]:
x, n = 2.01, 2
exr_math_trunc(x, n), exr_math_trunc(x, n, str_inp=False), exr_math_trunc(x, n, str_inp=True)

In [ ]:
for n in range(0, 5):
    for i in range(10):
        for j in range(10**n):
            sx = f"{i}.{j :0{n}d}1"
            #x = math.nextafter(float(sx), 0)
            x = float(sx)
            nonp = exr_math_trunc(x, n + 1, rounding=decimal.ROUND_DOWN)
            p = exr_math_trunc(x, n + 1, str_inp=False, rounding=decimal.ROUND_DOWN)
            if p != nonp:
                print(f"{sx=} {n=} {x=} {p=} {nonp=}")
                if sx > "0.00269":
                    break

In [ ]:
assert False

In [ ]:
%timeit exr_decimal_round(2012.5)
%timeit exr_round(2012.5)

In [ ]:
%timeit exr_decimal_round(2012.5, 0)
%timeit exr_round(2012.5, 0)

In [ ]:
%timeit assert True

In [ ]:
import fractions
exr_round(fractions.Fraction('3/2'), str_inp=False)

In [ ]:
import math
import numpy as np


In [ ]:
import numpy as np
exr_round("xx", 5, rounding=decimal.ROUND_DOWN, str_inp=False)

In [ ]:
assert False

In [ ]:
math.trunc(3.3)

In [ ]:
import math
import numpy as np

def check(rf):
    itest = [ 2.25, math.nextafter(2.5, 0), 2.5, 2.75, 3.25, math.nextafter(3.5, 0), 3.5, 3.75]
    otest = ["2",                 "2",     "3", "3",  "3",                 "3",     "4", "4"]
    for i1, o1 in zip(itest, otest):
        for i, o in zip([i1, -i1], [o1, "-" + o1]):
            r = rf(i)
            assert r == type(r)(o), f"i={i}, o={o}, r={r}, rf={rf}"
            assert len(str(r)) <= 4, f"{r=} {len(str(r))=} {str(r)=}"

def checkn(rf):
    itest = [ 2.25, math.nextafter(2.5, 0), 2.5, 2.75, 3.25, math.nextafter(3.5, 0), 3.5, 3.75]
    otest = ["2",                 "2",     "3", "3",  "3",                 "3",     "4", "4"]
    for i1, o1 in zip(itest, otest):
        for i, o in zip([i1, -i1], [o1, "-" + o1]):
            r = rf(i, 0)
            assert r == type(r)(o), f"i={i}, n=0, o={o}, r={r}, rf={rf}"
            assert len(str(r)) <= 4, f"{r=} {len(str(r))=} {str(r)=}"
    itest = [ 2.225, math.nextafter(2.25, 0),  2.25,  2.275, 
              3.325, math.nextafter(3.35, 0),  3.35,  3.375]
    otest = ["2.2",                "2.2",     "2.3", "2.3",  
             "3.3",                "3.3",     "3.4", "3.4"]
    for i1, o1 in zip(itest, otest):
        for i, o in zip([i1, -i1], [o1, "-" + o1]):
            r = rf(i, 1)
            if not np.allclose(float(r), float(o)):
                print(f"{i=}, n=1, pretty=False, o={o}, r={r}, rf={rf}")
            assert len(str(r)) <= 4, f"{i=}, n=1, pretty=False, {r=} {len(str(r))=} {str(r)=}"
            r = rf(i, 1, pretty=True)
            if not np.allclose(float(r), float(o)):
                print(f"i={i}, n=1, pretty=True, o={o}, r={r}, rf={rf}")
            assert len(str(r)) <= 4, f"{r=} {len(str(r))=} {str(r)=}"
    itest = [ 225,   math.nextafter(250, 0),   250,   275,
              325,   math.nextafter(350, 0),   350,   375]
    otest = ["200",                "200",     "300", "300", 
             "300",                "300",     "400", "400"]
    for i1, o1 in zip(itest, otest):
        for i, o in zip([i1, -i1], [o1, "-" + o1]):
            r = rf(i, -2)
            assert r == type(r)(o), f"i={i}, n=-2, o={o}, r={r}, rf={rf}"
            assert len(str(r)) <= 6, f"{r=} {len(str(r))=} {str(r)=}"
    itest = [ 2.675,  0.0000005, 0.15]
    stest = ["2.67", "0.000000", "0.1"]
    otest = ["2.68", "0.000001", "0.2"]
    ntest = [2,       6,         1]
    for i1, s1, o1, n in zip(itest, stest, otest, ntest):
        for i, s, o in zip([i1, -i1], [s1, "-" + s1], [o1, "-" + o1]):
            r = rf(i, n)
            if not np.allclose(float(r), float(s)):
                print(f"i={i}, n={n}, pretty=False, o={o}, r={r}, rf={rf}")
            assert len(str(r)) <= 9, f"{r=} {len(str(r))=} {str(r)=}"
            r = rf(i, n, pretty=True)
            assert r == type(r)(o), f"i={i}, n={n}, pretty=True, o={o}, r={r}, rf={rf}"
            assert len(str(r)) <= 9, f"{r=} {len(str(r))=} {str(r)=}"


In [ ]:
import decimal

_drtta_q0 = decimal.Decimal("1")

def decimal_roundTiesToAway(x, ndigits=0, pretty=False,
                            pretty_delta_x = float.fromhex('0x1.8p-1')  # 0.75ulp
                           ):
    """
    decimal_roundTiesToAway(x, n) - округлённое до n-цифр машинное представление x
    str(decimal_roundTiesToAway(x, n, pretty=True) - округлённое до n-цифр str(x)
    """
    dx = decimal.Decimal(x)
    if not ndigits:
        return dx.quantize(_drtta_q0, decimal.ROUND_HALF_UP)
    # assert math.ulp(x)/math.fabs(dx - dx.next_toward(0)) >= 2, \
    #     "Default context precision too small"
    q = decimal.Decimal(f"1e{-ndigits}")
    if not pretty or isinstance(x, str) or isinstance(x, decimal.Decimal):
        return dx.quantize(q, decimal.ROUND_HALF_UP)
    return (dx + 
            decimal.Decimal(math.copysign(pretty_delta_x*math.ulp(x), x))
           ).quantize(q, decimal.ROUND_HALF_UP)

In [ ]:
check(decimal_roundTiesToAway)
checkn(decimal_roundTiesToAway)

In [ ]:
import decimal

_drtta_q0 = decimal.Decimal("1")

def strdecimal_roundTiesToAway(x, ndigits=0, pretty=False):
    """
    strdecimal_roundTiesToAway(x, n) - округлённое до n-цифр машинное представление x
    str(strdecimal_roundTiesToAway(x, n, pretty=True) - округлённое до n-цифр str(x)
    """
    if not ndigits:
        return decimal.Decimal(x).quantize(_drtta_q0, decimal.ROUND_HALF_UP)
    # assert math.ulp(x)/math.fabs(dx - dx.next_toward(0)) >= 2, \
    #     "Default context precision too small"
    q = decimal.Decimal(f"1e{-ndigits}")
    if not pretty or isinstance(x, str) or isinstance(x, decimal.Decimal):
        return decimal.Decimal(x).quantize(q, decimal.ROUND_HALF_UP)
    return decimal.Decimal(str(x)).quantize(q, decimal.ROUND_HALF_UP)

In [ ]:
check(strdecimal_roundTiesToAway)
checkn(strdecimal_roundTiesToAway)

In [ ]:
(repr(2.0115), repr(decimal.Decimal(2.0115)),
 decimal_roundTiesToAway(2.0115, 3), 
 decimal_roundTiesToAway("2.0115", 3),
 decimal_roundTiesToAway(decimal.Decimal("2.0115"), 3))

In [ ]:
import fractions

def fractions_roundTiesToAway(x, ndigits=0, pretty=False,
                            pretty_delta_x = float.fromhex('0x1.8p-1')  # 0.75ulp
                           ):
    fx = fractions.Fraction(x)
    if not ndigits:
        return math.trunc(fx + fractions.Fraction(math.copysign(0.5, x)))
    if ndigits < 0:
        q = 10**-ndigits
        return math.trunc(fx/q + fractions.Fraction(math.copysign(0.5, x)))*q
    q = 10**ndigits
    if not pretty:
        return math.trunc(fx*q + fractions.Fraction(math.copysign(0.5, x)))/q
    return math.trunc((fx + 
                       fractions.Fraction(math.copysign(pretty_delta_x*math.ulp(x), x))
                      )*q +
                      fractions.Fraction(math.copysign(0.5, x))
                     )/q

In [ ]:
check(fractions_roundTiesToAway)
checkn(fractions_roundTiesToAway)

In [ ]:
import math

try:
    from math import fma as lc_fma  # Python 3.13 и выше
except ImportError:
    import ctypes
    import sys

    if sys.platform.startswith("darwin"):
        _libc = ctypes.CDLL("libSystem.B.dylib")
    elif sys.platform.startswith('win'):
        _libc = ctypes.cdll.msvcrt
    else:
        _libc = ctypes.CDLL("libc.so.6")
        
    _libc.fma.argtypes = [ctypes.c_double, ctypes.c_double, ctypes.c_double]
    _libc.fma.restype = ctypes.c_double
    lc_fma = _libc.fma

assert lc_fma(math.nextafter(1., 0), 
              math.nextafter(1., 0), 
              -(1. - math.ulp(1.))) == math.ulp(math.ulp(0.25))

def math_roundTiesToAway(x, ndigits=0, pretty=False,
                         pretty_delta_x = float.fromhex('0x1.p-1')  # 0.5ulp
                        ):
    """
    math_roundTiesToAway(x, n) - округлённое до n-цифр машинное представление x
    str(math_roundTiesToAway(x, n, pretty=True) - округлённое до n-цифр str(x)
    """
    if not ndigits:
        return math.trunc(x + math.copysign(0.5, x))
    if ndigits < 0:
        q = float(10**-ndigits)
        return math.trunc(x/q + math.copysign(0.5, x))*q
    q = float(10**ndigits)
    if not pretty:
        return math.trunc(lc_fma(x, q, math.copysign(0.5, x)))/q
    xl = math.copysign(pretty_delta_x*math.ulp(x), x)
    h = x + xl
    l = xl - (h - x)
    x, xl = h, l
    # assert x == x + xl  and  abs(xl) < math.ulp(x)
    ql = float(10**ndigits - int(q))
    # assert q == (q + ql)  and  abs(ql) < math.ulp(q)
    h = x*q
    l = lc_fma(x, q, -h)
    l += ql*x + xl*q 
    xq = h + l
    xql = l - (xq - h)
    # assert xq == (xq + xql)  and  abs(xql) < math.ulp(xq)
    a05 = math.copysign(0.5, x)
    h = xq + a05
    t = h - xq
    l = (xq - (h - t)) + (a05 - t)
    l += xql
    xqa05 = h + l
    xqa05l = l - (xqa05 - h)
    # assert xqa05 == (xqa05 + xqa05l)  and  abs(xqa05l) < math.ulp(xqa05)
    txqa05 = math.trunc(xqa05)
    if txqa05 == xqa05:
        txqa05 += math.floor(xqa05l) if 0 < xqa05 else math.ceil(xqa05l)
    return txqa05/q

In [ ]:
math_roundTiesToAway(3.35, 1, pretty=True)

In [ ]:
math_roundTiesToAway(math.nextafter(3.35, 0), 1, pretty=True)

In [ ]:
check(math_roundTiesToAway)
checkn(math_roundTiesToAway)

In [ ]:
lc_fma

In [ ]:
import numpy as np

def np_roundTiesToAway(x, ndigits=0, pretty=False,
                       pretty_delta_x = float.fromhex('0x1.p-1'),  # 0.5ulp
                       debug_double_double = False
                      ):
    if not ndigits:
        return np.trunc(x + np.copysign(0.5, x))
    if ndigits < 0:
        q = 10**-ndigits
        return np.trunc(x/q + np.copysign(0.5, x))*q
    q = 10**ndigits
    if not pretty:
        r = np.trunc(np.array([q, 1], dtype=type(x)) @
                        np.vstack((x, np.copysign(0.5, x)))
                       )/q
        return r if isinstance(x, np.ndarray) else r[0]
    ax = np.asarray(x)
    if np.half == ax.dtype:
        ext_t = np.float
    elif np.float32 == ax.dtype:
        ext_t = np.double
    elif np.double == ax.dtype and "longdouble" in np.__dict__:
        ext_t = np.longdouble
    else:
        ext_t = None
    if ext_t and not debug_double_double:
        ex = np.array(ax, dtype=ext_t)
        ex += np.copysign(np.finfo(ax.dtype).eps*pretty_delta_x*np.abs(ax), ax)
        r = np.trunc(np.array([q, 1], dtype=ex.dtype) @
                     np.vstack((ex, np.copysign(0.5, ex))),
                     dtype=ax.dtype
                    )/q
        # print(f"{x=} {type(x)=} {isinstance(x, np.ndarray)=}")
        return r if isinstance(x, np.ndarray) else r[0]
    assert False, "not implement"
    xl = np.copysign(pretty_delta_x*np.ulp(x), x)
    h = x + xl
    l = xl - (h - x)
    x, xl = h, l
    # assert x == x + xl  and  abs(xl) < math.ulp(x)
    q = type(x[0])(q)
    ql = type(x[0])(10**ndigits - int(q))
    # assert q == (q + ql)  and  abs(ql) < math.ulp(q)
    h = x*q
    l = np.fma(x, q, -h)
    l += ql*x + xl*q 
    xq = h + l
    xql = l - (xq - h)
    # assert xq == (xq + xql)  and  abs(xql) < math.ulp(xq)
    a05 = np.copysign(0.5, x)
    h = xq + a05
    t = h - xq
    l = (xq - (h - t)) + (a05 - t)
    l += xql
    xqa05 = h + l
    xqa05l = l - (xqa05 - h)
    # assert xqa05 == (xqa05 + xqa05l)  and  abs(xqa05l) < math.ulp(xqa05)
    txqa05 = np.trunc(xqa05)
    if txqa05 == xqa05:
        txqa05 += np.floor(xqa05l) if 0 < xqa05 else np.ceil(xqa05l)
    return txqa05/q

In [ ]:
x = 2.115
x = np.asarray(x)
x.dtype, np.finfo(x.dtype).eps, type(x)

In [ ]:
np_roundTiesToAway(3.35, 1)

In [ ]:
np_roundTiesToAway(3.35, 1, pretty=True)

In [ ]:
np_roundTiesToAway(math.nextafter(3.35, 0), 1, pretty=True)

In [ ]:
check(np_roundTiesToAway)
checkn(np_roundTiesToAway)

In [ ]:
tsx = [ "0.15", "0.145", "2.675", "2.0115", "0.0000005", ]
tdecimals = [1, 2, 2, 3, 6]
for sx, decimals in zip(tsx, tdecimals):
    x = float(sx)
    print(f"| {sx:9} | {x.hex():21}| ≈{decimal.Decimal(x):<21.17g} | {decimals:2}  | {decimal_roundTiesToAway(x, ndigits=decimals):<8} | {math_roundTiesToAway(x, ndigits=decimals):<19} | {np_roundTiesToAway(x, ndigits=decimals):<19} |")

In [ ]:
tsx = [ "0.15", "0.145", "2.675", "2.0115", "0.0000005", ]
tdecimals = [1, 2, 2, 3, 6]
for sx, decimals in zip(tsx, tdecimals):
    x = float(sx)
    print(f"| {sx:9} | {x.hex():21}| ≈{decimal.Decimal(x):<21.17g} | {decimals:2}  | {decimal_roundTiesToAway(x, ndigits=decimals, pretty=True):<8} | {math_roundTiesToAway(x, ndigits=decimals, pretty=True):<19} | {np_roundTiesToAway(x, ndigits=decimals, pretty=True):<19} |")

In [ ]:
tsx = [ "0.15", "0.145", "2.675", "2.0115", "0.0000005", ]
tdecimals = [1, 2, 2, 3, 6]
for sx, decimals in zip(tsx, tdecimals):
    x = float(sx)
    print(f"| {sx:9} | {x.hex():21}| ≈{decimal.Decimal(x):<21.17g} | {decimals:2}  | {decimal_roundTiesToAway(x, ndigits=decimals):<8} | {math_roundTiesToAway(x, ndigits=decimals):<19} | {fractions_roundTiesToAway(x, ndigits=decimals):<19} |")

In [ ]:
tsx = [ "0.15", "0.145", "2.675", "2.0115", "0.0000005", ]
tdecimals = [1, 2, 2, 3, 6]
for sx, decimals in zip(tsx, tdecimals):
    x = float(sx)
    print(f"| {sx:9} | {x.hex():21}| ≈{decimal.Decimal(x):<21.17g} | {decimals:2}  | {decimal_roundTiesToAway(x, ndigits=decimals, pretty=True):<8} | {math_roundTiesToAway(x, ndigits=decimals, pretty=True):<19} | {fractions_roundTiesToAway(x, ndigits=decimals, pretty=True):<19} |")

In [ ]:
math_roundTiesToAway(2.0115, 3)

In [ ]:
math_roundTiesToAway(2.0115, 3, pretty=True)

In [ ]:
decimal.Decimal(2.0115)

In [ ]:
for n in range(18):
    x = float("0." + n*"0" + "5")
    print(n, x, 
          float(math_roundTiesToAway(x, n)), 
          float(decimal_roundTiesToAway(x, n)),
          float(fractions_roundTiesToAway(x, n)))

In [ ]:
for n in range(3):
    for i in range(10):
        for j in range(10**n):
            sx = f"{i}.{j :0{n}d}5"
            x = float(sx)
            nonp = math_roundTiesToAway(x, n)
            p = math_roundTiesToAway(x, n, pretty=True)
            if p != nonp:
                print(f"{sx=} {n=} {x=} {p=} {nonp=}")

In [ ]:
for n in range(5):
    for i in range(10):
        for j in range(10**n):
            sx = f"{i}.{j :0{n}d}5"
            x = float(sx)
            pa = math_roundTiesToAway(x, n)
            pan = math_roundTiesToAway(x, n, pretty=True)
            pb = fractions_roundTiesToAway(x, n)
            pbn = fractions_roundTiesToAway(x, n, pretty=True)
            if pb == pbn and float(pa) != float(pb):
                print(f"{sx=} {n=} {x=} {pa=} {pb=}")
            if pb == pbn and float(pan) != float(pbn):
                print(f"{sx=} {n=} {x=} {pan=} {pbn=}")

In [ ]:
f"{3 :03d}"

In [ ]:
assert False

In [ ]:
print(f"{fractions_roundTiesToAway(2.0115, 3)=}")
%timeit fractions_roundTiesToAway(2.0115, 3)
print(f"{decimal_roundTiesToAway(2.0115, 3)=}")
%timeit decimal_roundTiesToAway(2.0115, 3)
print(f"{strdecimal_roundTiesToAway(2.0115, 3)=}")
%timeit strdecimal_roundTiesToAway(2.0115, 3)
print(f"{math_roundTiesToAway(2.0115, 3)=}")
%timeit math_roundTiesToAway(2.0115, 3)

In [ ]:
print(f"{fractions_roundTiesToAway(2.0115, 3, pretty=True)=}")
%timeit fractions_roundTiesToAway(2.0115, 3, pretty=True)
print(f"{decimal_roundTiesToAway(2.0115, 3, pretty=True)=}")
%timeit decimal_roundTiesToAway(2.0115, 3, pretty=True)
print(f"{strdecimal_roundTiesToAway(2.0115, 3, pretty=True)=}")
%timeit strdecimal_roundTiesToAway(2.0115, 3, pretty=True)
print(f"{math_roundTiesToAway(2.0115, 3, pretty=True)=}")
%timeit math_roundTiesToAway(2.0115, 3, pretty=True)

In [ ]:
print(f"{decimal_roundTiesToAway(2.675, 2)=}")
%timeit decimal_roundTiesToAway(2.675, 2)
print(f"{math_roundTiesToAway(2.675, 2)=}")
%timeit math_roundTiesToAway(2.675, 2)

In [ ]:
print(f"{decimal_roundTiesToAway(2.675, 2, pretty=True)=}")
%timeit decimal_roundTiesToAway(2.675, 2, pretty=True)
print(f"{math_roundTiesToAway(2.675, 2, pretty=True)=}")
%timeit math_roundTiesToAway(2.675, 2, pretty=True)

In [ ]:
x = 2.675
a0 = np.double(x)
a1000 = np.full(1000, a0)
q = decimal.Decimal("1.")
%timeit math_roundTiesToAway(x)
%timeit float(math_roundTiesToAway(x))
# %timeit np_roundTiesToAway(a0)
# %timeit np_roundTiesToAway(a1000)
# %timeit roundTiesToAway(x)
%timeit decimal_roundTiesToAway(x)
%timeit decimal.Decimal(x).quantize(q, decimal.ROUND_HALF_UP)

In [ ]:
if isinstance(a1000, np.half):
    ext_t = np.float
elif isinstance(a1000, np.half):
    ext_t = np.double
elif ((isinstance(a1000, np.double) or 
       isinstance(a1000, float)
      ) and "longdouble" in np.__dict__):
    ext_t = np.longdouble
else:
    ext_t = None
type(a0) == np.double

In [ ]:
b0.dtype, a0.dtype

In [ ]:
"longdouble" in np.__dict__

In [ ]:
x = math.nextafter(3.35, 0)
x = 3.35
ndigits = 1
q = float(10**ndigits)
lc_fma(x, q, math.copysign(0.5, x)), x*q + math.copysign(0.5, x)

In [ ]:
lc_fma(x, q, -x*q)/math.ulp(x*q)

In [ ]:
q.hex()

In [ ]:
2.225 + (math.ulp(2.225)/2)

In [ ]:
pretty_delta_x = float.fromhex('0x1.p-1')
x = 2.225
xl = math.copysign(pretty_delta_x*math.ulp(x), x)
h = x + xl
l = xl - (h - x)
assert h == (h + l)

In [ ]:
math_roundTiesToAway(2.675, 2)

In [ ]:
x = 2.0115
print(f"{int(x)=}")
%timeit int(x)
print(f"{float(int(x))=}")
%timeit float(int(x))
print(f"{math.trunc(x)=}")
%timeit math.trunc(x)
print(f"{float(math.trunc(x))=}")
%timeit float(math.trunc(x))
print(f"{math.modf(x)[1]=}")
%timeit math.modf(x)[1]
print(f"{x//1.=}")
%timeit x//1.
print(f"{x//math.copysign(1., x)=}")
%timeit x//math.copysign(1., x)
print(f"{(x//1. if x >= 0 else x//-1.)=}")
%timeit (x//1. if x >= 0 else x//-1.)
print(f"{")

In [ ]:
print(f"{round(x)=}")
%timeit round(x)
print(f"{round(x, 0)=}")
%timeit round(x, 0)

In [ ]:
math.inf//1.

In [ ]:
x.as_integer_ratio()

In [ ]:
type(round(x, -2))

In [ ]:
round(math.nan, -2)

In [ ]:
9223372036854775807. .as_integer_ratio()

In [ ]:
f"{9223372036854775807. :.16g}"

In [ ]:
def sum(a, b):
    return str(a) + str(b)
# xprint = print
# xprint = sum
xprint = lambda *x: None
%timeit xprint(f"{math.sin(math.pi)}", "")

In [ ]:
import logging
import math
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
logger.info('Doing something')
%timeit logger.debug(f"{math.sin(math.pi)}")
%timeit logger.debug(f" math.sin(math.pi) ")

In [ ]:
import fractions
fractions.Fraction(3.35)

In [ ]:
math.trunc(fractions.Fraction(3.35)*10)

In [ ]:
def cfrac_aprox(x):
    

In [ ]:
import fractions 
x = 0.3
# fractions.Fraction(x)
num, den = x.as_integer_ratio()
num, den, divmod(num, den)

In [ ]:
x = np.float32(0.3)
num, den = x.as_integer_ratio()
print(f"  {num=} {den=}")
d, r = divmod(num, den)
print(f"{d=} {r=}")
num, den = den, r
print(f"  {num=} {den=}")
d, r = divmod(num, den)
print(f"{d=} {r=}")
num, den = den, r
print(f"  {num=} {den=}")
d, r = divmod(num, den)
print(f"{d=} {r=}")
num, den = den, r
print(f"  {num=} {den=}")
d, r = divmod(num, den)
print(f"{d=} {r=}")
num, den = den, r
print(f"  {num=} {den=}")
d, r = divmod(num, den)
print(f"{d=} {r=}")
num, den = den, r
print(f"  {num=} {den=}")
d, r = divmod(num, den)
print(f"{d=} {r=}")

In [ ]:
#x = np.float32(0.3)
x = 0.3
num, den = x.as_integer_ratio()
print(f"{num, den=}")
cr = []
while 0 < den:
    (d, den), num = divmod(num, den), den
    cr.append(d)
print(cr)

In [ ]:
for i in range(1, len(cr)):
    n, d = 1, 0
    for a in cr[i::-1]:
        n, d = a*n + d, n
    print(f"{n, d=} {n/d=}")

In [ ]:
def cf_approx(x):
    num0, den0 = x.as_integer_ratio()
    num, den = num0, den0
    cr = []
    while 0 < den:
        (d, den), num = divmod(num, den), den
        cr.append(d)
        n, d = 1, 0
        for a in cr[-1::-1]:
            n, d = a*n + d, n
        if n/d == x:
            return n, d
    return num0, den0

In [ ]:
cf_approx(300.1)

In [ ]:
tsx = [ "0.15", "0.145", "2.675", "2.0115", "0.0000005" ]
for t in tsx:
    print(f"{t=} {cf_approx(float(t))=}")

In [ ]:
cf_approx(0.55555555555555555555555)

In [ ]:
_crtta_12 = [fractions.Fraction(-1, 2), 
             fractions.Fraction(1, 2)]

def cf_roundTiesToAway(x, ndigits=0, pretty=True):
    if ndigits < 0:
        q = 10**-ndigits
        return math.trunc(fractions.Fraction(*cf_approx(x))/q + _crtta_12[x >= 0])*q
    q = 10**ndigits
    return math.trunc(fractions.Fraction(*cf_approx(x))*q + _crtta_12[x >= 0])/q
    
check(cf_roundTiesToAway)
checkn(cf_roundTiesToAway)

In [ ]:
fractions.Fraction(1, 3)/7

In [ ]:
tsx = [ "0.15", "0.145", "2.675", "2.0115", "0.0000005", ]
tdecimals = [1, 2, 2, 3, 6]
for sx, decimals in zip(tsx, tdecimals):
    x = float(sx)
    print(f"| {sx:9} | {x.hex():21}| ≈{decimal.Decimal(x):<21.17g} | {decimals:2}  | {decimal_roundTiesToAway(x, ndigits=decimals, pretty=True):<8} | { cf_roundTiesToAway(x, ndigits=decimals):<19} |")

In [ ]:
print(f"{cf_roundTiesToAway(2.0115, 3)=}")
%timeit cf_roundTiesToAway(2.0115, 3)
print(f"{fractions_roundTiesToAway(2.0115, 3)=}")
%timeit fractions_roundTiesToAway(2.0115, 3)
print(f"{decimal_roundTiesToAway(2.0115, 3)=}")
%timeit decimal_roundTiesToAway(2.0115, 3)
print(f"{math_roundTiesToAway(2.0115, 3)=}")
%timeit math_roundTiesToAway(2.0115, 3)

In [ ]:
print(f"{cf_roundTiesToAway(2.0115, 3, pretty=True)=}")
%timeit cf_roundTiesToAway(2.0115, 3, pretty=True)
print(f"{fractions_roundTiesToAway(2.0115, 3, pretty=True)=}")
%timeit fractions_roundTiesToAway(2.0115, 3, pretty=True)
print(f"{decimal_roundTiesToAway(2.0115, 3, pretty=True)=}")
%timeit decimal_roundTiesToAway(2.0115, 3, pretty=True)
print(f"{math_roundTiesToAway(2.0115, 3, pretty=True)=}")
%timeit math_roundTiesToAway(2.0115, 3, pretty=True)

In [ ]:
def fcf_bin_ratio(x):
    n, d = x.as_integer_ratio()
    return float(n), float(d)

In [ ]:
def fcf_ratio(x):
    num0, den0 = fcf_bin_ratio(x)
    num, den = num0, den0
    cr = []
    while 0 < den:
        (d, den), num = divmod(num, den), den
        cr.append(d)
        n, d = 1, 0
        for a in cr[-1::-1]:
            n, d = a*n + d, n
        if n/d == x:
            return n, d
    return num0, den0

In [ ]:
def fcf_mul(a, b):
    return a[0]*b[0], a[1]*b[1]

def fcf_adds12(a):
    # return a[0] + a[1]*math.copysign(0.5, a[0]), a[1]
    return math.fma(a[1], math.copysign(0.5, a[0]), a[0]), a[1]

def fcf_modf(a):
    if 0 <= a[0]:
        i, n = divmod(a[0], a[1])
        return i, (n, a[1])
    else:
        i, n = divmod(-a[0], a[1])
        return -i, (-n, a[1])
    
def fcf_trunc(a):
    return math.trunc(a[0]/a[1])

assert fcf_mul(fcf_ratio(0.), (1, 10)) == (0.0, 10)
assert fcf_mul(fcf_ratio(0.3), (10, 1)) == (30.0, 10.0)
assert fcf_mul(fcf_ratio(-0.3), (10, 1)) == (-30.0, 10.0)
assert fcf_adds12(fcf_ratio(1/3)) == (2.5, 3.0)
assert fcf_adds12(fcf_ratio(-1/3)) == (-2.5, 3.0)
assert fcf_modf(fcf_ratio(1/3)) == (0.0, (1.0, 3.0))
assert fcf_modf(fcf_ratio(-1/3)) == (0.0, (-1.0, 3.0))
assert fcf_modf(fcf_ratio(4/3)) == (1.0, (1.0, 3.0))
assert fcf_modf(fcf_ratio(-4/3)) == (-1.0, (-1.0, 3.0))
assert fcf_trunc(fcf_ratio(1/3)) == 0
assert fcf_trunc(fcf_ratio(-2/3)) == 0
assert fcf_trunc(fcf_ratio(4/3)) == 1
assert fcf_trunc(fcf_ratio(-4/3)) == -1

In [ ]:
def fcf_roundTiesToAway(x, ndigits=0, pretty=True):
    ratio = fcf_ratio(x)
    if ndigits < 0:
        q = 10**-ndigits
        i, f = fcf_modf(fcf_mul(fcf_ratio(x), (1, q)))
        return (i + fcf_trunc(fcf_adds12(f)))*q
    q = 10**ndigits
    i, f = fcf_modf(fcf_mul(fcf_ratio(x), (q, 1)))
    return (i + fcf_trunc(fcf_adds12(f)))/q

check(fcf_roundTiesToAway)
checkn(fcf_roundTiesToAway)

In [ ]:
tsx = [ "0.15", "0.145", "2.675", "2.0115", "0.0000005", ]
tdecimals = [1, 2, 2, 3, 6]
for sx, decimals in zip(tsx, tdecimals):
    x = float(sx)
    print(f"| {sx:9} | {x.hex():21}| ≈{decimal.Decimal(x):<21.17g} | {decimals:2}  | {decimal_roundTiesToAway(x, ndigits=decimals, pretty=True):<8} | { fcf_roundTiesToAway(x, ndigits=decimals):<19} |")

In [ ]:
print(f"{fcf_roundTiesToAway(2.0115, 3)=}")
%timeit fcf_roundTiesToAway(2.0115, 3)
print(f"{fractions_roundTiesToAway(2.0115, 3)=}")
%timeit fractions_roundTiesToAway(2.0115, 3)
print(f"{decimal_roundTiesToAway(2.0115, 3)=}")
%timeit decimal_roundTiesToAway(2.0115, 3)
print(f"{math_roundTiesToAway(2.0115, 3)=}")
%timeit math_roundTiesToAway(2.0115, 3)

In [ ]:
print(f"{fcf_roundTiesToAway(2.0115, 3, pretty=True)=}")
%timeit fcf_roundTiesToAway(2.0115, 3, pretty=True)
print(f"{fractions_roundTiesToAway(2.0115, 3, pretty=True)=}")
%timeit fractions_roundTiesToAway(2.0115, 3, pretty=True)
print(f"{decimal_roundTiesToAway(2.0115, 3, pretty=True)=}")
%timeit decimal_roundTiesToAway(2.0115, 3, pretty=True)
print(f"{math_roundTiesToAway(2.0115, 3, pretty=True)=}")
%timeit math_roundTiesToAway(2.0115, 3, pretty=True)

In [ ]:
divmod(5., 3.)

In [ ]:
-1.0 - 2/3, -5/3

In [ ]:
x = 249.99999999999997
ndigits = -2
q = 10**-ndigits
fcf_ratio(x)

In [ ]:
fcf_mul(fcf_ratio(x), (1, q))

In [ ]:
8796093022207999.0/3518437208883200.0

In [ ]:
divmod(*fcf_mul(fcf_ratio(x), (1, q)))

In [ ]:
1759218604441599.0/3518437208883200.0

In [ ]:
(1759218604441599.0 + 0.5*3518437208883200.0)/3518437208883200.0

In [ ]:
lc_fma(0.5, 3518437208883200.0, 1759218604441599.0)/3518437208883200.0

In [ ]:
z=fcf_adds12(fcf_mul(fcf_ratio(x), (1, q)))

In [ ]:
(z[0]/z[1]).hex()

In [ ]:
35184372088832/2/2

In [ ]:
LOG = False
%timeit LOG and print(f"{math.sin()}")

In [ ]:
LGF = lambda: False
%timeit LGF() and print(f"{math.sin()}")

In [ ]:
isinstance("xxx", str)

In [ ]:
tsx = [ "0.15", "0.145", "2.675", "2.0115", "0.0000005", ]
for sx in tsx:
    for x in [float(sx), -float(sx)]:
        print(decimal.Decimal(x))
        print(str(x), f"{x :.16g}")


In [ ]:
decimal.Decimal?

In [ ]:
decimal.getcontext()

In [ ]:
decimal.getcontext().create_decimal_from_float(2.0115)

In [ ]:
decimal.getcontext().prec

In [ ]:
1 - decimal.Decimal(1).next_toward()

In [ ]:
1 - decimal.Decimal(10)**-decimal.getcontext().prec

In [ ]:
from decimal import Decimal, ROUND_HALF_UP
x = 5.465
float(Decimal(str(x)).quantize(Decimal('1.00'), ROUND_HALF_UP))

In [ ]:
str(str(x))

In [ ]:
str(np.longdouble(str(x)))

In [ ]:
x = np.array([5.465]*10)

In [ ]:
ldx = np.longdouble(x)

In [ ]:
ldx, ldx[0]

In [ ]:
np.longdouble(5.465)

In [ ]:
np.str

In [ ]:
np.array_repr(x)

In [ ]:
np.fromstring(np.array_repr(ldx))

In [ ]:
repr(5.465)

In [ ]:
np.longdouble(str(ldx[0]))

In [ ]:
fractions.Fraction(5465, 100).__round__(2)

In [ ]:
float(round(fractions.Fraction(5475, 1000), 2))

In [ ]:
import numbers
x = decimal.Decimal("2.675")
for x in [ 2.675, "2.675", 
          decimal.Decimal("2.675"), fractions.Fraction("2.675"),
          #np.longdouble("2.675")
         ]:
    print(repr(x), 
          ((round(x, 2)
               if isinstance(x, numbers.Number) else None),
           round(fractions.Fraction(x), 2),
           (round(fractions.Fraction(*x.as_integer_ratio()), 2) 
               if isinstance(x, numbers.Number) else None),
           (round(decimal.Decimal(x), 2))
               if not isinstance(x, fractions.Fraction) else None),
         )

In [ ]:
np.longdouble("2.675")

In [ ]:
round(decimal.Decimal(2.675), 2)

In [ ]:
import sys
# sys.float_repr_style = 'legacy'
# sys.float_repr_style = 'short'
print(f"{sys.float_repr_style=} default 'short', _PY_SHORT_FLOAT_REPR == 1")
for x in [math.pi, -math.pi, np.nan, np.inf, 
          1., -1., 2.675, -2.675,
          math.pi*1e300, math.pi*1e-300, 
          math.nextafter(0., math.inf), math.nextafter(math.inf, 0.),
          math.pi*math.nextafter(0., math.inf), math.nextafter(math.inf, 0.)/math.pi,
          0., -0.]:
    print(str(x) == repr(x), str(x), repr(x))


In [ ]:
assert lc_fma(math.nextafter(1., 0), 
              math.nextafter(1., 0), 
              -(1. - math.ulp(1.))) == math.ulp(math.ulp(0.25))

In [ ]:
a = np.array([0.999, 1.])
b = np.array([0.999, -(1. - 0.002)])
a @ b

In [ ]:
a = np.array([math.nextafter(1., 0), 1.])
b = np.array([math.nextafter(1., 0), -(1. - math.ulp(1.))])
a @ b

In [ ]:
np.dot(a, b)

In [ ]:
np.show_config()

In [ ]:
np.lib.introspect.opt_func_info?

In [ ]:
dict = np.lib.introspect.opt_func_info(
...     func_name="add|abs", signature="float64|complex64"
... )

In [ ]:
import json
print(json.dumps(dict, indent=2))

In [ ]:
for n in range(4, 5):
    for i in range(10):
        for j in range(10**n):
            sx = f"{i}.{j :0{n}d}1"
            #x = math.nextafter(float(sx), 0)
            x = float(sx)
            nonp = decimal.Decimal(x).quantize(decimal.Decimal(f"1e-{n + 1}"),
                                rounding=decimal.ROUND_DOWN)
            p = decimal.Decimal(str(x)).quantize(decimal.Decimal(f"1e-{n + 1}"),
                                rounding=decimal.ROUND_DOWN)
            if p != nonp:
                print(f"{sx=} {n=} {x=} {p=} {nonp=}")
                if sx > "0.00269":
                    break

In [ ]:
x = 0.00261
n = 5
decimal.Decimal(x).quantize(decimal.Decimal(f"1e-{n}"),
                                rounding=decimal.ROUND_DOWN)

In [ ]:
import decimal
import math 

for b in [0.00161, 0.00261, 4.00001, 4.10001, float(0.15+0.005*62)]:
    for j in [math.nextafter(b, 0), b, math.nextafter(b, math.inf)]:
        machine = str(decimal.Decimal(j).quantize(
                        decimal.Decimal("0.00001"),
                        rounding=decimal.ROUND_DOWN))
        human = str(decimal.Decimal(str(j)).quantize(
                        decimal.Decimal("0.00001"),
                        rounding=decimal.ROUND_DOWN))
        print(f"j={j:<22}", machine, human, 
              "machine != human" if machine != human else "")

In [ ]:
str(decimal.Decimal(str(j=float(0.15+0.005*62float(0.15+0.005*62))).quantize(
        decimal.Decimal("0.00001"),
        rounding=decimal.ROUND_DOWN))

In [ ]:
j = 4.00001 ; print(j - j % 1e-5)
j = 0.00261 ; print(j - j % 1e-5)

In [ ]:
j=float(0.15+0.005*62) ; print(j - j % 1e-5)

In [ ]:
def to_str(x, i):
    n, d = x.as_integer_ratio()
    s = str(n * 10 ** i // d)
    j = len(s) - i
    if j > 0:
        return f'{s[:j]}.{s[j:]}'
    return f'0.{"0" * -j}{s}'
    
to_str(4.00001, 5), to_str(0.00261, 5)

In [ ]:
to_str(4.00001, 5), to_str(4.10001, 5), to_str(0.00261, 5), to_str(0.00161, 5)

In [ ]:
fractions.Fraction(str(0.3)), fractions.Fraction(str(1/3))

In [ ]:
str(x)

In [ ]:
import ctypes
import sys

if sys.platform.startswith("darwin"):
    _libc = ctypes.CDLL("libSystem.B.dylib")
elif sys.platform.startswith('win'):
    _libc = ctypes.cdll.msvcrt
else:
    _libc = ctypes.CDLL("libc.so.6")
 

In [ ]:
x = 2.675
n = 2
round(x, n), float(f"{x :.{n}f}"), float(f"{x :.{n + 1}g}")

In [ ]:
_libc.printf(b"%f\n", ctypes.c_double(x))

In [ ]:
x = _libc.printf(b"An int %d, a double %f\n", 1234, ctypes.c_double(3.14))
x

In [ ]:
round("2")

In [ ]:
round(2**51 + 0.15, 1)

In [ ]:
exr_round(2**50 + 1.15, 1, str_inp=False, 
    rounding=decimal.ROUND_HALF_EVEN)

In [ ]:
for b in [0.00161, 0.00261, 4.00001, 4.10001, float(0.15+0.005*62)]:
    print(f"{b:<22}{b : .5f}")

In [ ]:
for n, x, hm in [(10, 524287.9999291525, 524287.9999291524),
                 (10, 262143.99998263217, 262143.9999826321)]:
    q = 10**n
    xf, xi = math.modf(x)
    r = xi + math.trunc(xf*q)/q
    print(math.trunc(xf*q), math.trunc(xf*q)/q, r)
    print(math.modf(r))
    print(math.modf(hm))
    print((x - hm)/math.ulp(hm))


In [ ]:
n = 10
x = 262143.99998263217
q = 10**n
xf, xi = math.modf(x)
r = xi + math.trunc(xf*q)/q
rf, ri = math.modf(r)
print(x, xf, xi)
print(r, rf, ri)
print(exr_decimal_round(x, n, rounding=decimal.ROUND_DOWN),
      (exr_decimal_round(x, n, rounding=decimal.ROUND_DOWN) - r),
      (exr_decimal_round(x, n, rounding=decimal.ROUND_DOWN) - r)/math.ulp(r))

In [ ]:
n = 10
x = 262143.99998263217
q = 10**n
xf, xi = math.modf(x)
r = xi + exr_sql_round_base(xf*q)/q
rf, ri = math.modf(r)
print(x, xf, xi)
print(r, rf, ri)
print(exr_decimal_round(x, n, rounding=decimal.ROUND_HALF_EVEN),
      (exr_decimal_round(x, n, rounding=decimal.ROUND_HALF_EVEN) - r),
      (exr_decimal_round(x, n, rounding=decimal.ROUND_HALF_EVEN) - r)/math.ulp(r))

In [ ]:
np.log2(26214399998263220 )

In [ ]:
r_rounding = decimal.ROUND_CEILING
def r(x):
    return math.ceil(x)

r_rounding = decimal.ROUND_FLOOR
def r(x):
    return math.floor(x)

r_rounding = decimal.ROUND_DOWN
def r(x):
    return math.trunc(x)

r_rounding = decimal.ROUND_HALF_UP
def r(x):
    return math.trunc(x + math.copysign(0.5, x))

r_rounding = decimal.ROUND_HALF_EVEN
def r(x):
    return round(x)

def a(x, n):
    q = 10**n
    return r(x*q)/q

def b(x, n):
    q = 10**n
    xf, xi = math.modf(x)
    return (xi*q + r(xf*q))/q

def c(x, n):
    q = 10**n
    xq = x*q
    xql = math.fma(x, q, -xq)
    txq = r(xq)
    if txq == xq:
        txq += r(xql)
    return txq/q

for x in [2.616, -2.616]:
    for n in range(4):
        e = [exr_easy_round(x, n, rounding=r_rounding, fmt_out=fo) 
             for fo in [True, False]]
        assert a(x, n) in e
        assert b(x, n) in e
        assert c(x, n) in e
print('Хорь')

In [ ]:
x, n, e, a(x, n), b(x, n), c(x, n)

In [ ]:
%timeit a(2.0115, 3)
%timeit b(2.0115, 3)
%timeit c(2.0115, 3)

In [ ]:
import sys
math.log10(2) * sys.float_info.mant_dig/2

In [ ]:
10**-4

In [ ]:
for t in [np.half, np.single, np.double, np.longdouble]:
    for n in range(5):
        t1 = t(1)/t(10**n)
        t1m = np.nextafter(t(t1), 0)
        t05 = t('0.' + n*'0' + '5')
        t05p = np.nextafter(t(t05), np.inf)
        t05m = np.nextafter(t(t05), 0)
        t01 = t('0.' + n*'0' + '1')
        cvt = exr_decimal_from_longdouble if t == np.longdouble else float
        lens = [len(decimal.Decimal(cvt(tx)).as_tuple().digits)
                  for tx in [t1m, t05p, t05, t05m, t01]]
        print(t, n, max(lens), lens)

In [ ]:
lfp = np.ceil(max(0, -np.log2(np.spacing(t1m))))
lip = np.ceil(max(0, np.log10(abs(t1m))))
lfp, lip

In [ ]:
def dlen(x):
    lfp = math.ceil(max(0, np.log10(abs(x)) - np.log2(np.spacing(x/10))))
    lip = math.ceil(max(0, np.log10(abs(x))))
    return lfp + lip, lfp, lip

In [ ]:
for t in [float, np.half, np.single, np.double, np.longdouble]:
    bs = [np.finfo(t).smallest_normal, np.finfo(t).smallest_subnormal, 
          np.finfo(t).max, t(0.5)]
    xs = [t(np.nextafter(b, d)) for b in bs 
          for d in [np.inf, b, -np.inf ] ]
    cvt = exr_decimal_from_longdouble if t == np.longdouble else float
    lens = [len(decimal.Decimal(cvt(x)).as_tuple().digits)
            for x in xs]
    print(f"{xs[0].__class__.__name__:10} {max(lens):6d} {lens=}")


In [ ]:
xs[0].__class__.__name__

In [ ]:
float(1).__class__.__name__

In [ ]:
[len(decimal.Decimal(cvt(x)).as_tuple().digits)
            for x in xs]

In [ ]:
[len(decimal.Decimal(cvt(x)).as_tuple().digits)
            for x in xs[5:7]]

In [ ]:
xs[5:7]

In [ ]:
xs[-1], cvt(xs[-1])

In [ ]:
math.isinf(xs[-1])

In [ ]:
dsmallest_subnormal = decimal.Decimal(np.finfo(t).smallest_subnormal)
len(dsmallest_subnormal.as_tuple().digits), dlen(np.finfo(t).smallest_subnormal)

In [ ]:
dmax = decimal.Decimal(np.finfo(t).max)
len(dmax.as_tuple().digits), dlen(np.finfo(t).max)

In [ ]:
dlen(dsmallest_normal)

In [ ]:
dsmallest_subnormal.as_tuple().digits[-10:]

In [ ]:
        lens = [len(str(decimal.Decimal(cvt(tx)))) - 2 - n 
                  for tx in [t1m, t05p, t05, t05m, t01]]
        print(t, n, max(lens), lens)
        lens = [len(decimal.Decimal(cvt(tx)).as_tuple().digits)
                  for tx in [t1m, t05p, t05, t05m, t01]]
        print(t, n, max(lens), lens)

In [ ]:
cvt(t1m)

In [ ]:
type(np.longdouble(t1m))

In [ ]:
t1m = np.nextafter(np.longdouble(1), 0)
np.spacing(t1m)
# Вызывает RuntimeWarning: invalid value encountered in spacing
#t1m = t1m/10
print(t1m)
# t1m = np.nextafter(np.nextafter(np.longdouble(1), 0), 0)
# print(t1m)
np.spacing(t1m), t1m, type(t1m)

In [ ]:
[(str(decimal.Decimal(float(tx))))
                  for tx in [t1m, t05p, t05, t05m, t01]]

In [ ]:
dt01 = decimal.Decimal(t01/1000)
len(dt01.as_tuple().digits), dt01.as_tuple()

In [ ]:
with decimal.localcontext(prec=100):
    t = np.float128
    t1m = np.nextafter(t(1), 0)
    t05p = np.nextafter(t(0.5), 2)
    t0125 = t(0.125)
    t0125m = np.nextafter(t(0.125), 0)
    t01p = np.nextafter(t('0.1'), 2)
    t01 = t('0.1')
    print(len(str(decimal.Decimal(exr_decimal_from_longdouble(t1m))))-2,
          len(str(decimal.Decimal(exr_decimal_from_longdouble(t05p))))-2,
          len(str(decimal.Decimal(exr_decimal_from_longdouble(t0125))))-2,
          len(str(decimal.Decimal(exr_decimal_from_longdouble(t0125m))))-2,
          len(str(decimal.Decimal(exr_decimal_from_longdouble(t01p))))-2,
          len(str(decimal.Decimal(exr_decimal_from_longdouble(t01))))-2,
         )

In [ ]:
dt01 = decimal.Decimal(exr_decimal_from_longdouble(t01/1000))

In [ ]:
len(dt01.as_tuple().digits), dt01.as_tuple()

In [ ]:
dt01

In [ ]:
t01, decimal.Decimal(exr_decimal_from_longdouble(np.longdouble('4.0001')))

In [ ]:
t01, decimal.Decimal((4.0001))

In [ ]:
with decimal.localcontext(prec=100):
    print(decimal.Decimal(1)/decimal.Decimal(10))

In [ ]:
decimal.Decimal(round(fractions.Fraction(20125, 10000),3))

In [ ]:
decimal.getcontext().prec

In [ ]:
import decimal
import fractions

def exr_approximate_decimal_from_integer_ratio(x, prec=None):
    with decimal.localcontext(prec=(prec if prec else
                                    decimal.getcontext().prec + 4)):
        n, d = x.as_integer_ratio()
        return decimal.Decimal(n)/decimal.Decimal(d)

def exr_decimal_to_fraction(d):
    return fractions.Fraction(*d.as_integer_ratio())

In [ ]:
exr_approximate_decimal_from_integer_ratio(
    round(fractions.Fraction('2.0125'),3))

In [ ]:
round(decimal.Decimal('2.0125'), 3)

In [ ]:
round(2.0125, 3)

In [ ]:
for t in [float, decimal.Decimal, fractions.Fraction]:
    print(t)
    for sx0, n0 in [('0.5', 0), ('1.5', 0), ('2.5', 0), 
                   ('0.05', 1), ('0.15', 1), ('0.25', 1), 
                   ('0.005', 2), ('0.015', 2), ('0.025', 2), 
                  ]:
        for n in ([n0] if n0 else [None, n0]): 
            for sx in ["-" + sx0, sx0]:
                x = t(sx)
                rx = round(x, n)
                frx = float(rx)
                drx = (rx if t != fractions.Fraction else
                       exr_approximate_decimal_from_integer_ratio(rx))
                print(f"{sx :<7} {x :<7} {str(n):4} -> {rx :-7} ({frx} {drx})")
          